In [ ]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from collections import defaultdict
import re
from matplotlib import colors as colors_mpl
import seaborn as sns
import zipfile
from CRISPResso2 import CRISPRessoShared
from CRISPResso2 import CRISPResso2Align

In [2]:
# Collab New Data - Compound Ref Alignment
crispresso_output_folder = "CRISPRessoBatch_on_aavs1_ccr5_compound_aln/CRISPResso_on_eeBxbI_50attP_AAVS1_PE2max_rep1/"
# crispresso_output_folder = "/CRISPRessoBatch_on_aavs1_ccr5_compound_aln/CRISPResso_on_eeBxbI_50attP_AAVS1_PE2max_rep2/"
# crispresso_output_folder = "/CRISPRessoBatch_on_aavs1_ccr5_compound_aln/CRISPResso_on_eeBxbI_50attP_AAVS1_PE2max_rep3/"
# crispresso_output_folder = "/CRISPRessoBatch_on_aavs1_ccr5_compound_aln/CRISPResso_on_eeBxbI_50attP_CCR5_PE2max_rep1/"
# crispresso_output_folder = "/CRISPRessoBatch_on_aavs1_ccr5_compound_aln/CRISPResso_on_eeBxbI_50attP_CCR5_PE2max_rep2/"
# crispresso_output_folder = "/CRISPRessoBatch_on_aavs1_ccr5_compound_aln/CRISPResso_on_eeBxbI_50attP_CCR5_PE2max_rep3/"
# AAVS1 TwinPE reference sequence
twin_seq = 'CGCCGGGAACTGCCGCTGGCCCCCCACCGCCAGGTTTGTCTGGTCAACCACCGCGGTCTCAGTGGTGTACGGTACAAACCTCCACTTCCAGGAGTCGCTGTGCCCCGATGCACACTGGGAAGTCCGCAGCTCCGAGGCGCCCAGTGGAAATCGCCAGATGAGGGCCTCCTC'
wt_seq =   'CGCCGGGAACTGCCGCTGGCCCCCCACCGCCCCAAGGATCTCCCGGTCCCCGCCCGGCGTGCTGACGTCACGGCGCTGCCCCAGGGTGTGCTGGGCAGGTCGCGGGGAGCGCTGGGAAATGGAGTCCATTAGCAGAAGTGGCCCTTGGCCACTTCCAGGAGTCGCTGTGCCCCGATGCACACTGGGAAGTCCGCAGCTCCGAGGCGCCCAGTGGAAATCGCCAGATGAGGGCCTCCTC'
# compound_ref_seq = 'CGCCGGGAACTGCCGCTGGCCCCCCACCGCCCCAAGGATCTCCCGGTCCCCGCCCGGCGTGCTGACGTCACGGCGCTGCCCCAGGGTGTGCTGGGCAGGTCGCGGGGAGCGCTGGGAAATGGAGTCCATTAGCAGAAGTGGCCCTTGGAGGTTTGTCTGGTCAACCACCGCGGTCTCAGTGGTGTACGGTACAAACCTCCACTTCCAGGAGTCGCTGTGCCCCGATGCACACTGGGAAGTCCGCAGCTCCGAGGCGCCCAGTGGAAATCGCCAGATGAGGGCCTCCTC'
# CCR5 TwinPE reference sequence
# twin_seq = 'ACAATCGATAGGTACCTGGCTGTCGTCCATGCTGTGTTTGCTTTAAAAGCCAGGACGGTCACCTTTGGGGTGGTGACAAGTGTGATCACTTGGGAGGTTTGTCTGGTCAACCACCGCGGTCTCAGTGGTGTACGGTACAAACCTGCTCTCATTTTCCATACAGTCAGTATCAATTCTGGAAGAATTTCCAGACATTAAAGATAGTCATCTTGGGGCTGGT'
# compound_ref_seq = 'ACAATCGATAGGTACCTGGCTGTCGTCCATGCTGTGTTTGCTTTAAAAGCCAGGACGGTCACCTTTGGGGTGGTGACAAGTGTGATCACTTGGGTGGTGGCTGTGTTTGCGTCTCTCCCAGGAATCATCTTTACCAGATCTCAAAAAGAAGGTCTTCATTACACCTGCAAGGTTTGTCTGGTCAACCACCGCGGTCTCAGTGGTGTACGGTACAAACCTGCTCTCATTTTCCATACAGTCAGTATCAATTCTGGAAGAATTTCCAGACATTAAAGATAGTCATCTTGGGGCTGGT'

In [3]:
# Anzalone Data

#### PE3 12bp ins sample
# crispresso_output_folder = "../04_crispresso_on_anzalone/CRISPRessoBatch_on_fig2b_hek3_pe3_ins_12bp/CRISPResso_on_Fig2b_HEK3_PE3_ins_12bp_rep1/"
# PE3 Reference Sequence for 12bp ins
# Full twin pe ref
# twin_seq = 'GCTGCCTAGAAAGGCATGGATGAGAGAAGCCTGGAGACAGGGATCCCAGGGAAACGCCCATGCAATTAGTCTATTTCTGCTGCAAGTAAGCATGCATTTGTAGGCTTGATGCTTTTTTTCTGCTTCTCCAGCCCTGGCCTGGGTCAATCCTTGGGGCCCAGACTGAGCACGGGAGTGCAGGTGTGATGGCAGAGGAAAGGAAGCCCTGCTTCCTCCAGAGGGCGTCGCAGGACAGCTTTTCCTAGACAGGGGCTAGTATGTGCAGCTCCTGCACCGGGATAC'
# Truncated twin pe ref - matches wt ref seq length
# twin_seq = 'GCTGCCTAGAAAGGCATGGATGAGAGAAGCCTGGAGACAGGGATCCCAGGGAAACGCCCATGCAATTAGTCTATTTCTGCTGCAAGTAAGCATGCATTTGTAGGCTTGATGCTTTTTTTCTGCTTCTCCAGCCCTGGCCTGGGTCAATCCTTGGGGCCCAGACTGAGCACGGGAGTGCAGGTGTGATGGCAGAGGAAAGGAAGCCCTGCTTCCTCCAGAGGGCGTCGCAGGACAGCTTTTCCTAGACAGGGGCTAGTATGTGCAGCTCCT'

#### PE3 108bp ins sample
# crispresso_output_folder = "../04_crispresso_on_anzalone/CRISPRessoBatch_on_fig2b_hek3_pe3_ins_108bp/CRISPResso_on_Fig2b_HEK3_PE3_ins_108bp_rep1/"
# # Full PE3 Ref Seq for 108bp ins
# twin_seq = 'GCTGCCTAGAAAGGCATGGATGAGAGAAGCCTGGAGACAGGGATCCCAGGGAAACGCCCATGCAATTAGTCTATTTCTGCTGCAAGTAAGCATGCATTTGTAGGCTTGATGCTTTTTTTCTGCTTCTCCAGCCCTGGCCTGGGTCAATCCTTGGGGCCCAGACTGAGCACGGGAGTGCAGGTGGAAACCATCTCCCCAGGAGACGGGCGCACCTTCCCCAAGCGCGGCCAGACCTGCGTGGTGCACTACACCGGGATGCTTGAAGATGGAAAGAAATTTTGATGGCAGAGGAAAGGAAGCCCTGCTTCCTCCAGAGGGCGTCGCAGGACAGCTTTTCCTAGACAGGGGCTAGTATGTGCAGCTCCTGCACCGGGATAC'

#### TwinPE 108bp ins sample
# crispresso_output_folder = "../04_crispresso_on_anzalone/CRISPRessoBatch_on_fig2b_hek3_twinpe_ins_108bp/CRISPResso_on_Fig2b_HEK3_twinPE_ins_108bp_rep1/"
# TwinPE Reference Sequence for 108bp ins
# twin_seq = 'CCCAGCCAAACTTGTCAACCAGTATCCCGGAAATTTCTTTCCATCTTCAAGCATCCCGGTGTAGTGCACCACGCAGGTCTGGCCGCGCTTGGGGAAGGTGCGCCCGTCTCCTGGGGAGATGGTTTCCACCTGCACTCCCGTGCTCAGTCTGGGCCCCAAGGATTGACCCAGGCCAGGGCTGGAGAAGCAGAAAAAAAGCATCAAGCCTACAAATGCATGCTTACTTGCAGCAGAAATAGACTAATTGCATGGGCGTTTCCCTGGGATCCCTGTCTCCAGG'

#### TwinPE 38bp ins sample Fig1c A38 B38
# crispresso_output_folder = "../04_crispresso_on_anzalone/CRISPRessoBatch_on_fig1c_twinpe_38bp_ins/CRISPResso_on_Fig1c_attB_A38_B38_twinPE_ins_38bp_rep1/"
# twin_seq = 'AAGGCATGGATGAGAGAAGCCTGGAGACAGGGATCCCAGGGAAACGCCCATGCAATTAGTCTATTTCTGCTGCAAGTAAGCATGCATTTGTAGGCTTGATGCTTTTTTTCTGCTTCTCCAGCCCTGGCCTGGGTCAATCCTTGGGGCCCAGACTGAGCACGGGCTTGTCGACGACGGCGGTCTCCGTCGTCAGGATCATCCGGGATACTGGTTGACAAGTTTGGCTGGG' # full length twinPE ref - shorter than WT Ref due to 90bp deletion but actually contains more of the original bases
# twin_seq = 'AAGGCATGGATGAGAGAAGCCTGGAGACAGGGATCCCAGGGAAACGCCCATGCAATTAGTCTATTTCTGCTGCAAGTAAGCATGCATTTGTAGGCTTGATGCTTTTTTTCTGCTTCTCCAGCCCTGGCCTGGGTCAATCCTTGGGGCCCAGACTGAGCACGGGCTTGTCGACGACGGCGGTCTCCGTCGTCAGGATCATCCGGGATAC' # Issues with finding Perfect PE so can't use - truncated twinPE ref to base match WT Ref - might help with alignment and visualization

In [4]:
def get_refpos_values(ref_aln_seq, read_aln_seq):
    """
    Given a reference alignment this returns a dictionary such that refpos_dict[ind] is the value of the read at the position corresponding to the ind'th base in the reference
    Any additional bases in the read (gaps in the ref) are assigned to the first position of the ref (i.e. refpos_dict[0])
    For other additional bases in the ref (gaps in the read), the value is appended to the last position of the ref that had a non-gap base (to the left)
    For example:
    ref_seq =  '--A-TGC-'
    read_seq = 'GGAGTCGA'
    get_refpos_values(ref_seq, read_seq)
    {0: 'GGAG', 1: 'T', 2: 'C', 3: 'GA'}
    Args:
    - ref_aln_seq: str, reference alignment sequence
    - read_aln_seq: str, read alignment sequence
    Returns:
    - refpos_dict: dict, dictionary such that refpos_dict[ind] is the value of the read at the position corresponding to the ind'th base in the reference
    """
    refpos_dict = defaultdict(str)

    # First, if there are insertions in read, add those to the first position in ref
    if ref_aln_seq[0] == '-':
        aln_index = 0
        read_start_bases = ""
        while aln_index < len(ref_aln_seq) and ref_aln_seq[aln_index] == '-':
            read_start_bases += read_aln_seq[aln_index]
            aln_index += 1
        refpos_dict[0] = read_start_bases
        ref_aln_seq = ref_aln_seq[aln_index:]
        read_aln_seq = read_aln_seq[aln_index:]
        
    ref_pos = 0
    last_nongap_ref_pos = 0
    for ind in range(len(ref_aln_seq)):
        ref_base = ref_aln_seq[ind]
        read_base = read_aln_seq[ind]
        if ref_base == '-':
            refpos_dict[last_nongap_ref_pos] += read_base
        else:
            refpos_dict[ref_pos] += read_base
            last_nongap_ref_pos = ref_pos
            ref_pos += 1
    return refpos_dict

In [5]:
# print(twin_aln_seq)
# print(comp_aln_seq_pe)
# print(comp_aln_seq_wt)
# print(wt_aln_seq)

In [20]:
print('Processing ' + crispresso_output_folder)
crispresso2_info = CRISPRessoShared.load_crispresso_info(crispresso_output_folder)

num_changes_to_check = 2

if 'Comp' not in crispresso2_info['results']['refs']:
    raise Exception('Reference sequence for reference "Comp" not found - was a reference sequence named "Comp" provided?')

if not twin_seq:
    raise Exception('Reference sequence for reference "TwinPE" not found - was a reference sequence named "TwinPE" provided?')

comp_ref_seq  = crispresso2_info['results']['refs']['Comp']['sequence']
comp_ref_name = crispresso2_info['results']['ref_names'][0]

# Create alignments using the same scoring parameters CRISPResso2
# used for read alignments to ensure consistent gap behavior.
aln_gap_incentive = crispresso2_info['results']['refs'][comp_ref_name]['gap_incentive']
aln_gap_open_arg = crispresso2_info['running_info']['args'].needleman_wunsch_gap_open
aln_gap_extend_arg = crispresso2_info['running_info']['args'].needleman_wunsch_gap_extend

aln_matrix_loc = crispresso2_info['running_info']['args'].needleman_wunsch_aln_matrix_loc
if aln_matrix_loc == 'EDNAFULL':
    aln_matrix = CRISPResso2Align.make_matrix()
else:
    if not os.path.exists(aln_matrix_loc):
        raise Exception('Alignment matrix file not found at ' + aln_matrix_loc)
    aln_matrix = CRISPResso2Align.read_matrix(aln_matrix_loc)

# Align TwinPE to Compound Ref Sequence
twin_aln_seq, comp_aln_seq_pe, aln_score = CRISPResso2Align.global_align(
    twin_seq, 
    comp_ref_seq, 
    matrix=aln_matrix,
    gap_incentive=aln_gap_incentive,
    gap_open=aln_gap_open_arg,
    gap_extend=aln_gap_extend_arg
)

# Align WT to Compound Ref Sequence
wt_aln_seq, comp_aln_seq_wt, aln_score = CRISPResso2Align.global_align(
    wt_seq, 
    comp_ref_seq, 
    matrix=aln_matrix,
    gap_incentive=aln_gap_incentive,
    gap_open=aln_gap_open_arg,
    gap_extend=aln_gap_extend_arg
)

# Alignment debugging check - may need to change bp_changes_arr range if alignments differ (use longest?)
if len(comp_aln_seq_pe) != len(comp_aln_seq_wt):
    raise Exception('Compound ref alignments to TwinPE and WT have different lengths!')

# Determine deletion and insertion regions in WT and TwinPE refs - these are the base changes
bp_changes_arr = []
del_start = del_end = ins_start = ins_end = None
for idx in range(len(comp_aln_seq_pe)):
    wt_base = wt_aln_seq[idx]
    twin_base = twin_aln_seq[idx]
    if wt_base != '-' and twin_base == '-':
        bp_changes_arr.append((idx, wt_base, twin_base))
        if del_start is None:
            del_start = idx
        del_end = idx
    elif wt_base == '-' and twin_base != '-':
        bp_changes_arr.append((idx, wt_base, twin_base))
        if ins_start is None:
            ins_start = idx
        ins_end = idx
del_region_coords = (del_start, del_end)
ins_region_coords = (ins_start, ins_end)

# Debugging checks
del_region_len = del_end - del_start + 1
ins_region_len = ins_end - ins_start + 1
if len(bp_changes_arr) != del_region_len + ins_region_len:
    raise Exception('Length of bp_changes_arr does not equal sum of del and ins region lengths.')

print(f'Found {str(len(bp_changes_arr))} base changes (via compound alignment): {str(bp_changes_arr)}')

# Map Compound ref bases to WT and TwinPE ref bases
wt_map = get_refpos_values(comp_aln_seq_wt, wt_aln_seq)
pe_map = get_refpos_values(comp_aln_seq_pe, twin_aln_seq)

# Load allele frequency table
z = zipfile.ZipFile(os.path.join(crispresso_output_folder, crispresso2_info['running_info']['allele_frequency_table_zip_filename']))
zf = z.open(crispresso2_info['running_info']['allele_frequency_table_filename'])
df_alleles = pd.read_csv(zf,sep="\t")

nuc_bases = ['A','C','G','T']

from_left_all_edit_counts = [0] * ins_region_len
from_right_all_edit_counts = [0] * ins_region_len

from_left_all_edit_counts_no_deletions = [0] * ins_region_len
from_left_all_edit_counts_with_deletions = [0] * ins_region_len
from_right_all_edit_counts_no_deletions = [0] * ins_region_len
from_right_all_edit_counts_with_deletions = [0] * ins_region_len
edit_counts_no_deletions = [0] * ins_region_len
edit_counts_with_deletions = [0] * ins_region_len

allele_counts = {} # e.g. TTRTT > 100
allele_categories = {} # e.g. TTRTT > Left_flap

edit_counts = [0] * ins_region_len
deletion_counts = [0] * ins_region_len
insertion_counts = [0] * ins_region_len
perfect_T_count = 0

total_alleles = 0
total_alleles_reads = 0
total_alleles_deletions_reads = 0

# cat_wt_count = 0
# cat_wt_count_arr = [0] * len(bp_changes_arr)

# cat_wt_indel_count = 0
# cat_wt_indel_count_arr = [0] * len(bp_changes_arr)

# cat_left_flap_count = 0
# cat_left_flap_count_arr = [0] * len(bp_changes_arr)

# cat_right_flap_count = 0
# cat_right_flap_count_arr = [0] * len(bp_changes_arr)

# cat_perfect_pe_count = 0
# cat_perfect_pe_count_arr = [0] * len(bp_changes_arr)

# cat_imperfect_pe_count = 0
# cat_imperfect_pe_count_arr = [0] * len(bp_changes_arr)

# cat_pe_indels_count = 0
# cat_pe_indels_count_arr = [0] * len(bp_changes_arr)

cat_wt_count = 0
cat_wt_count_arr = [0] * ins_region_len

cat_wt_indel_count = 0
cat_wt_indel_count_arr = [0] * ins_region_len

cat_left_flap_count = 0
cat_left_flap_count_arr = [0] * ins_region_len

cat_right_flap_count = 0
cat_right_flap_count_arr = [0] * ins_region_len

cat_perfect_pe_count = 0
cat_perfect_pe_count_arr = [0] * ins_region_len

cat_imperfect_pe_count = 0
cat_imperfect_pe_count_arr = [0] * ins_region_len

cat_pe_indels_count = 0
cat_pe_indels_count_arr = [0] * ins_region_len

has_indel_in_match_arr_count = 0
has_any_indel_byproduct_count = 0

# iterate all alleles in input allele table
for idx, allele in df_alleles.iterrows():
    total_alleles += 1
    total_alleles_reads += allele['#Reads']

    comp_aln_seq_read = allele.Reference_Sequence
    read_aln_seq = allele.Aligned_Sequence

    # Map this reads aligned bases to Compound ref aligned bases 
    read_map = get_refpos_values(comp_aln_seq_read, read_aln_seq)

    # Map this alleles match_arr index to ref_seq index
    match_arr_to_comp_base_index = {}
    # Build an array showing how this allele matches Comp vs TwinPE at each changed position.
    # 'W' = matches WT base
    # 'T' = matches TwinPE base
    # 'D' = non-programmed deletion
    # 'I' = non-programmed insertion
    # 'S' = non-programmed substitution
    # 'N' = ambiguous base
    # Build an array for both the deletion and inserted regions (bp_changes_arr)
    match_arr = []
    for ind, (comp_ind, wt_base, twin_base) in enumerate(bp_changes_arr):
        base_at_pos = read_map.get(comp_ind, "")
        if base_at_pos == wt_base:
            match_arr.append("W")
        elif base_at_pos == twin_base:
            match_arr.append("T")
        elif base_at_pos == "-":
            match_arr.append("D")
        # include insertions in PE_indel category if it still has all the edits?
        # elif len(base_at_pos) > 1:
        elif len(base_at_pos) > 1 and base_at_pos[0] == twin_base:
            match_arr.append("T")
            # match_arr.append("I")
        elif base_at_pos in nuc_bases and base_at_pos != wt_base and base_at_pos != twin_base:
            match_arr.append("S")
        else:
            match_arr.append("N")
        match_arr_to_comp_base_index[ind] = comp_ind

    # Array for only the sequence that is deleted by TwinPE
    del_match_arr = match_arr[:del_end-del_start+1]
    # Array for only the sequence that is inserted by TwinPE
    ins_match_arr = match_arr[del_end-del_start+1:]

    # # Build an array for both the deletion and inserted regions
    # match_arr = [0] * len(bp_changes_arr)
    # for ind, (comp_ind, wt_base, twin_base) in enumerate(bp_changes_arr):
    #     base_at_pos = read_map.get(comp_ind, "")
    #     if base_at_pos == wt_base:
    #         match_arr[ind] = "W"
    #     elif base_at_pos == twin_base:
    #         match_arr[ind] = "T"
    #     elif base_at_pos == "-":
    #         match_arr[ind] = "D"
    #     elif len(base_at_pos) > 1:
    #         match_arr[ind] = "I"
    #     else:
    #         match_arr[ind] = "N"
    #     match_arr_to_ref_base_index[ind] = comp_ind
    
    # # indels anywhere in the allele
    # has_any_indel = False
    # if allele.all_deletion_positions != '[]': # need to run with detailed allele output
    #     has_any_indel = True
    # if allele.all_insertion_positions != '[]':
    #     has_any_indel = True

    # # indels within quantification window
    # has_indel_guide = False
    # if allele.n_deleted > 0:
    #     has_indel_guide = True
    # if allele.n_inserted > 0:
    #     has_indel_guide = True

    # has_indel = has_indel_guide

    # check match_arr for indels
    has_indel_in_match_arr = any(m in ["I", "D"] for m in match_arr)
    if has_indel_in_match_arr:
        has_indel_in_match_arr_count += 1
    # check allele table for indels anywhere
    # check crispresso insertion list first
    has_any_indel_byproduct = False
    if allele.all_insertion_left_positions != "[]":
        has_any_indel_byproduct = True
    # check crispresso deletion list if needed - if deletion idx is not within programmed indel region
    else:
        edit_range = range(del_start, ins_end + 1)
        all_del_pos = [int(x) for x in allele.all_deletion_positions.strip("[]").split(",")]
        for del_ind in all_del_pos:
            if del_ind not in edit_range:
                has_any_indel_byproduct = True
                break
    if has_any_indel_byproduct:
        has_any_indel_byproduct_count += 1
    
    # has_indel = False
    # if has_any_indel_byproduct or has_indel_in_match_arr:
    has_indel = has_any_indel_byproduct
    if has_indel:
        total_alleles_deletions_reads += allele['#Reads']

    match_arr_str = "\t".join(match_arr) + "\t" + str(has_indel)
    if match_arr_str not in allele_counts:
        allele_counts[match_arr_str] = 0
    allele_counts[match_arr_str] += allele['#Reads']

    # Define PE sites -> every position that differs between WT and PE

    # 1st break:
    # WT vs Prime edited

    # If every PE site is PE 
    # If it has indels -> PE indel (indel anywhere - including outside of guide windows)
    # if not -> perfect PE

    # If every PE site is WT
    # If it has indels - >WT indel (anywhere, even outside)
    # if not, -> WT

    # If left two PE sites are PE:
    # Left flap
    # If right two PE sites are PE:
    # Right flap
    # Otherwise, 
    # Imperfect PE

    total_TE_count = match_arr.count("T")
    has_all_TE = (total_TE_count == len(match_arr))

    total_WT_count = match_arr.count("W")
    has_all_WT = (total_WT_count == len(match_arr))

    # ins_TE_count = ins_match_arr.count("T")
    # is_imperfect_PE = (ins_TE_count >= num_changes_to_check and ins_TE_count < len(ins_match_arr))

    # define left/right flap booleans
    is_left_flap = False
    is_right_flap = False
    if len(ins_match_arr) >= num_changes_to_check:
        is_left_flap = all(m == "T" for m in ins_match_arr[:num_changes_to_check])
        is_right_flap = all(m == "T" for m in ins_match_arr[-num_changes_to_check:])
        # is_left_flap = all(m == "T" for m in match_arr[del_end-del_start+1:del_end-del_start+1+num_changes_to_check])
        # is_right_flap = all(m == "T" for m in match_arr[-num_changes_to_check:])

    # is_left_flap = True
    # for i in range(num_changes_to_check):
    #     if len(match_arr) > 1:
    #         if match_arr[i] != 'T':
    #             is_left_flap = False

    # is_right_flap = True
    # for i in range(num_changes_to_check):
    #     if len(match_arr) > 1:
    #         if match_arr[-1-i] != 'T':
    #             is_right_flap = False

    # TE_count = 0
    # for match_val in match_arr:
    #     if match_val == 'T':
    #         TE_count += 1

    # is_perfect_PE = False
    # if TE_count == len(match_arr):
    #     is_perfect_PE = True
    
    # is_imperfect_PE = False
    # if TE_count > num_changes_to_check:
    #     is_imperfect_PE = True

    category = "NA"
    if has_all_TE and has_indel:
        category = "PE_Indel"
        df_alleles.at[idx,'Category'] = category
        cat_pe_indels_count += allele['#Reads']
        for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
            if match == 'T':
                cat_pe_indels_count_arr[pos_idx] += allele['#Reads']
    
    elif has_all_TE:
        category = "Perfect_PE"
        df_alleles.at[idx,'Category'] = category
        cat_perfect_pe_count += allele['#Reads']
        for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
            if match == 'T':
                cat_perfect_pe_count_arr[pos_idx] += allele['#Reads']

    elif has_all_WT and has_indel:
        category = "WT_Indel"
        df_alleles.at[idx,'Category'] = category
        cat_wt_indel_count += allele['#Reads']
        for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
            if match == 'T':
                cat_wt_indel_count_arr[pos_idx] += allele['#Reads']

    elif has_all_WT:
        category = "WT"
        df_alleles.at[idx,'Category'] = category
        cat_wt_count += allele['#Reads']
        for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
            if match == 'T':
                cat_wt_count_arr[pos_idx] += allele['#Reads']

    elif is_left_flap and not is_right_flap:
        category = "Left_Flap"
        df_alleles.at[idx,'Category'] = category
        cat_left_flap_count += allele['#Reads']
        for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
            if match == 'T':
                cat_left_flap_count_arr[pos_idx] += allele['#Reads']

    elif is_right_flap and not is_left_flap:
        category = "Right_Flap"
        df_alleles.at[idx,'Category'] = category
        cat_right_flap_count += allele['#Reads']
        for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
            if match == 'T':
                cat_right_flap_count_arr[pos_idx] += allele['#Reads']

    else:
        category = "Imperfect_PE"
        df_alleles.at[idx,'Category'] = category
        cat_imperfect_pe_count += allele['#Reads']
        for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
            if match == 'T':
                cat_imperfect_pe_count_arr[pos_idx] += allele['#Reads']

    if match_arr_str in allele_categories and allele_categories[match_arr_str] != category:
        raise Exception('conflicting categories for ' + match_arr_str + ' ' + category + ' vs ' + allele_categories[match_arr_str])
    allele_categories[match_arr_str] = category

    for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
        if match == 'T':
            from_left_all_edit_counts[pos_idx] += allele['#Reads']

            if has_indel:
                from_left_all_edit_counts_with_deletions[pos_idx] += allele['#Reads']
            else:
                from_left_all_edit_counts_no_deletions[pos_idx] += allele['#Reads']

        else:
            break

    for pos_idx, match in zip(reversed(range(len(ins_match_arr))), reversed(ins_match_arr)):
        if match == 'T':
            from_right_all_edit_counts[pos_idx] += allele['#Reads']

            if has_indel:
                from_right_all_edit_counts_with_deletions[pos_idx] += allele['#Reads']
            else:
                from_right_all_edit_counts_no_deletions[pos_idx] += allele['#Reads']
        else:
            break

    for pos_idx, match in zip(range(len(ins_match_arr)), ins_match_arr):
        if match == 'T':
            edit_counts[pos_idx] += allele['#Reads']
            if has_indel:
                edit_counts_with_deletions[pos_idx] += allele['#Reads']
            else:
                edit_counts_no_deletions[pos_idx] += allele['#Reads']
        if match == 'D':
            deletion_counts[pos_idx] += allele['#Reads']
        if match == "I":
            insertion_counts[pos_idx] += allele["#Reads"]
    if ins_match_arr == ['T']*len(ins_match_arr):
        perfect_T_count += allele['#Reads']

    #### DEBUG ####
    # # if read_aln == 'CGCCGGGAACTGCCGCTGGCCCCCCACCGCC---AGGTTTGTCTGGTCAACCACCGCGGT-------------------CTCAGTGGTGTACGGTACAAACCTC------CACTTCCA---GGAGTCGGTCAACCACCGCGGTCTCAGTGGTGTACGGTACAAACCTCCACTTCCAGGAGTCGCTGTGCCCCGATGCACACTGGGAAGTCCGCAGCTCCGAGGCGCCCAGTGGAAATCGCCAGATGAGGGCCTCCTC':
    # if 'CGCCGGGAACTGCCGCTGGCCCCCCACCGCC---------------------------------------------------------------------------------------------------------------------AGGTTTGTCTGGTCAACCACCGCGGTCTCAGTGGTGTACGGTACAAACCTCCACTTCCAGGAGTCGCTGTGCCCCGATGCACACTGGGAAGTCCGCAGCT----GGCCCCCCACCGCCAGGTTTCGCCAGATGAGGGCCTCCTC' in read_aln_seq:
    #     print(f"{allele['#Reads'] =  }")
    #     # print(f"{allele['Category'] =  }")
    #     print(f"{category =          }")
    #     print(f"{wt_aln_seq =        }")
    #     print(f"{twin_aln_seq =      }")
    #     print(f"{comp_aln_seq_pe =   }")
    #     print(f"{comp_aln_seq_read = }")
    #     print(f"{read_aln_seq =      }")
    #     print(f"{all_del_pos =                            }")
    #     print(f"{allele['all_deletion_positions'] =       }")
    #     print(f"{allele['all_insertion_left_positions'] = }")
    #     print(f"{bp_changes_arr = }")
    #     print(f"{match_arr =      }")
    #     print(f"{del_match_arr =  }")
    #     print(f"{ins_match_arr =  }")
    #     print(f"{del_region_coords = }")
    #     print(f"{ins_region_coords = }")
    #     print(f"{match_arr_to_comp_base_index = }")
    #     print(f"{read_map = }")
    #     print(f"{wt_map =   }")
    #     print(f"{pe_map =   }")
    #     print(f"{has_indel =  }")
    #     print(f"{has_all_WT = }")
    #     print(f"{has_all_TE = }")
    #     # print(allele.all_insertion_positions)
    #     # print(allele.all_insertion_left_positions)

imperfect_from_left_all_edit_counts = [x-perfect_T_count for x in from_left_all_edit_counts] #these are reads that start with a twin-edited, but are not completely twin-edited. Note that from_left_all_edit_counts includes perfect matches
imperfect_from_right_all_edit_counts = [x-perfect_T_count for x in from_right_all_edit_counts]
perfect_edit_counts = [perfect_T_count] * ins_region_len
total_counts = [total_alleles_reads] * ins_region_len

# with open('01_classify_reads_7cat.ipynb.allele_counts.txt','w') as fout:
#     sorted_allele_counts = sorted(allele_counts.keys(), key=lambda x: allele_counts[x], reverse=True)
#     fout.write("\t".join([str(x) for x in bp_changes_arr]) + '\thas_indel\tcount\n')
#     for allele_str in sorted_allele_counts:
#         fout.write(allele_str + '\t' + str(allele_counts[allele_str]) + '\n')
        
# with open('01_classify_reads_7cat.ipynb.arrays.txt','w') as fout:
#     fout.write('Class\t' + "\t".join([str(x) for x in bp_changes_arr]) + '\n')
#     fout.write('total_counts\t' + "\t".join([str(x) for x in total_counts]) + '\n')
#     fout.write('edit_counts\t' + "\t".join([str(x) for x in edit_counts]) + '\n')
#     fout.write('perfect_edit_counts\t' + "\t".join([str(x) for x in perfect_edit_counts]) + '\n')
#     fout.write('from_left_all_edit_counts\t' + "\t".join([str(x) for x in from_left_all_edit_counts]) + '\n')
#     fout.write('from_right_all_edit_counts\t' + "\t".join([str(x) for x in from_right_all_edit_counts]) + '\n')
#     fout.write('imperfect_from_left_all_edit_counts\t' + "\t".join([str(x) for x in imperfect_from_left_all_edit_counts]) + '\n')
#     fout.write('imperfect_from_right_all_edit_counts\t' + "\t".join([str(x) for x in imperfect_from_right_all_edit_counts]) + '\n')
#     fout.write('deletion_counts\t' + "\t".join([str(x) for x in deletion_counts]) + '\n')

# with open('01_classify_reads_7cat.ipynb.counts.txt','w') as fout:
#     fout.write("\t".join(['WT','WT_indels','left_flap','right_flap','perfect_PE','imperfect_PE','PE_indels']) + '\n')
#     fout.write("\t".join([str(x) for x in [cat_wt_count, cat_wt_indel_count, cat_left_flap_count, cat_right_flap_count, cat_perfect_pe_count, cat_imperfect_pe_count, cat_pe_indels_count]]) + '\n')


print('Read ' + str(total_alleles) + ' alleles with ' + str(total_alleles_reads) + ' reads')
print('Got ' + str(total_alleles_deletions_reads) + ' alleles with deletions')
print('Got ' + str(perfect_T_count) + ' perfect T alleles')
# print(f"{has_indel_in_match_arr_count = }")
# print(f"{has_any_indel_byproduct_count = }")

Processing CRISPRessoBatch_on_aavs1_ccr5_compound_aln/CRISPResso_on_eeBxbI_50attP_AAVS1_PE2max_rep1/
Found 167 base changes (via compound alignment): [(31, 'C', '-'), (32, 'C', '-'), (33, 'A', '-'), (34, 'A', '-'), (35, 'G', '-'), (36, 'G', '-'), (37, 'A', '-'), (38, 'T', '-'), (39, 'C', '-'), (40, 'T', '-'), (41, 'C', '-'), (42, 'C', '-'), (43, 'C', '-'), (44, 'G', '-'), (45, 'G', '-'), (46, 'T', '-'), (47, 'C', '-'), (48, 'C', '-'), (49, 'C', '-'), (50, 'C', '-'), (51, 'G', '-'), (52, 'C', '-'), (53, 'C', '-'), (54, 'C', '-'), (55, 'G', '-'), (56, 'G', '-'), (57, 'C', '-'), (58, 'G', '-'), (59, 'T', '-'), (60, 'G', '-'), (61, 'C', '-'), (62, 'T', '-'), (63, 'G', '-'), (64, 'A', '-'), (65, 'C', '-'), (66, 'G', '-'), (67, 'T', '-'), (68, 'C', '-'), (69, 'A', '-'), (70, 'C', '-'), (71, 'G', '-'), (72, 'G', '-'), (73, 'C', '-'), (74, 'G', '-'), (75, 'C', '-'), (76, 'T', '-'), (77, 'G', '-'), (78, 'C', '-'), (79, 'C', '-'), (80, 'C', '-'), (81, 'C', '-'), (82, 'A', '-'), (83, 'G', '-'), (

In [ ]:
tot_reads = 0
dff = df_alleles[df_alleles['Category'] == 'PE_indels']
for i, row in dff.iterrows():
    if 'CGCCGGGAACTGCCGCTGGCCCCCCACCGCC---------------------------------------------------------------------------------------------------------------------AGGTTTGTCTGGTCAACCACCGCGGTCTCAGTGGTGTACGGTACAAACCTCCACTTCCAGGAGTCGCTGTGCCCCGATGCACACTG' in row['Aligned_Sequence']:
        print(i, row['#Reads'], row['Aligned_Sequence'])        
        tot_reads += row['#Reads']
print(f'Total PE_indels reads with specific sequence: {tot_reads}')

In [ ]:
for i, row in dff.sort_values(by='#Reads', ascending=False).head(25).iterrows():
    print(i, row)

In [ ]:
# print(wt_aln_seq)
# print(wt_ref_aln_seq)
# print(ref_aln_seq)
# print(twin_aln_seq)

In [ ]:
len(df_alleles)

In [7]:
df_alleles.Category.value_counts().sum()

1779

In [21]:
df_alleles.Category.value_counts()

Category
Imperfect_PE    640
WT              402
Right_Flap      305
Perfect_PE      283
PE_Indel         80
WT_Indel         53
Left_Flap        16
Name: count, dtype: int64

In [ ]:
df_alleles.Category.value_counts()

In [ ]:
# sss = 'CCCAGCCAAACTTGTCAACCAGTATCCCGGCAGACTGAGCACGGGAGTGCAGGTGGAAACCATCTCCCCAGGAGACGGGCGCACCTTCCCCAAGCGCGGCCAGACCTGCGTGGTCCAGTATCCCGGAAATTTCTTTCCATCTTCAAGCATCCCGGTGTAGTGCACCACGCAGGTCTGGCCGCGCTTGGGGAAGGTTGGGCCCCAAGGATTGACCCAGGCCAGGGCTGGAGAAGCAGAAAAAAAGCATCAAGCCTACAAATGCATGCTTACTTGCAGCAGAAATAGACTAATTGCATGGGCGTTTCCCTGGGATCCCTGTCTCCAGGCTTCTCTCATCCATGCCT'
# ins_108_A ='ACCATCTCCCCAGGAGACGGGCGCACCTTCCCCAAGCGCGGCCAGACCTGCGTGGTCCAGTATCCCGGAAATTTCTTTCCATCTTCAAGCATCCCGGTGTAGTGCACC'
# ins_108_B ='CCATCTCCCCAGGAGACGGGCGCACCTTCCCCAAGCGCGGCCAGACCTGCGTGGTCCAGTATCCCGGAAATTTCTTTCCATCTTCAAGCATCCCGGTGTAGTGCACCA'
for i, row in df_alleles.sort_values(by='#Reads', ascending=False).head(25).iterrows():
    print(row['#Reads'])
    print(row['Reference_Sequence'])
    # print(len(row['Reference_Sequence']))
    print(row['Aligned_Sequence'])
    print(row['Category'])
    print(row.all_insertion_left_positions)
    print(row.all_insertion_positions)
    print(row.all_deletion_positions)
    # print(len(row['Aligned_Sequence']))
    # if pe3_108_ref_seq_substring in row['Aligned_Sequence'].replace('-',''):
    # print(df_alleles.sort_values(by='#Reads', ascending=False).iloc[i]['Reference_Sequence'])
    # print(df_alleles.sort_values(by='#Reads', ascending=False).iloc[i]['Aligned_Sequence'])
    # AAATTTCTTTCCATCTTCAAGCATCCCGGTGTAGTGCACCACGCAGGTCTGGCCGCGCTTGGGGAAGGTGCGCCCGTCTCCTGGGGAGATGGTTTCCACCTGCACTCC

In [ ]:
def setBarMatplotlibDefaults():
    # matplotlib.use('AGG')
    # font = {'size': 22}
    # matplotlib.rc('font', **font)
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
    matplotlib.rcParams["font.sans-serif"] = ["Arial", "Liberation Sans", "Bitstream Vera Sans"]
    matplotlib.rcParams["font.family"] = "sans-serif"
    matplotlib.rcParams['axes.facecolor'] = 'white'
    # plt.ioff()

setBarMatplotlibDefaults() 

In [9]:
fig_root = 'figures/02_final_classify_reads_7cat/'

In [ ]:
# Indices for the x-axis
indices = np.arange(len(edit_counts))

# Plotting
fig, ax = plt.subplots(figsize=(14, 6)) 

# Stacked bar plot
ax.bar(indices, cat_perfect_pe_count_arr, label='Perfect PE')
bottom_so_far = cat_perfect_pe_count_arr

ax.bar(indices, cat_left_flap_count_arr, label='Left Flap', bottom=bottom_so_far)
bottom_so_far = [x + y for x, y in zip(bottom_so_far, cat_left_flap_count_arr)]

ax.bar(indices, cat_right_flap_count_arr, label='Right Flap', bottom=bottom_so_far)
bottom_so_far = [x + y for x, y in zip(bottom_so_far, cat_right_flap_count_arr)]

ax.bar(indices, cat_imperfect_pe_count_arr, label='Imperfect PE', bottom=bottom_so_far)
bottom_so_far = [x + y for x, y in zip(bottom_so_far, cat_imperfect_pe_count_arr)]

ax.bar(indices, cat_pe_indels_count_arr, label='PE Indels', bottom=bottom_so_far)
bottom_so_far = [x + y for x, y in zip(bottom_so_far, cat_pe_indels_count_arr)]

# By definition these cannot have any 'successful twin edits' so not including
# ax.bar(indices, cat_wt_indel_count_arr, label='WT Indels', bottom=bottom_so_far)
# bottom_so_far = [x + y for x, y in zip(bottom_so_far, cat_wt_indel_count_arr)]

# ax.bar(indices, cat_wt_count_arr, label='WT', bottom=bottom_so_far)
# bottom_so_far = [x + y for x, y in zip(bottom_so_far, cat_wt_count_arr)]

# Adding labels and title
ax.set_xlabel('Compound Reference Index')
ax.set_ylabel('Read Counts')
ax.set_title('Stacked Successful Twin Edits by Category')

ax.set_xticks(indices)
ax.set_xticklabels([str(x) for x in range(ins_start, ins_end+1)], rotation=45, ha='center')
# Dynamic? >35 x-tick = font 8 & rotate 45
ax.tick_params(axis='x', labelsize=8)
ax.legend()
plt.tight_layout()


# Display the plot
plt.savefig(fig_root + '.successful_twin_edit_counts_by_category.pdf')

# Display the plot
plt.show()

# # Display or save
# if fig_filename_root is None:
#     plt.show()
# else:
#     fig.savefig(fig_filename_root+'.pdf', bbox_inches='tight', bbox_extra_artists=(lgd,))
#     if SAVE_ALSO_PNG:
#         fig.savefig(fig_filename_root+'.png', bbox_inches='tight', bbox_extra_artists=(lgd,))
# plt.close(fig)

In [ ]:
deletion_counts

In [ ]:
# Indices for the x-axis
indices = np.arange(len(edit_counts))

# Plotting
fig, ax = plt.subplots(figsize=(14, 6)) 

# Stacked bar plot
ax.bar(indices, total_counts, label='Total Counts')
ax.bar(indices, edit_counts, label='Total TE Edit Counts')
ax.bar(indices, from_right_all_edit_counts, label='From Right All TE Edit Counts')
ax.bar(indices, from_left_all_edit_counts, label='From Left All TE Edit Counts')
ax.bar(indices, perfect_edit_counts, label='Perfect TE Edit Counts')
ax.bar(indices, deletion_counts, label='Deletion Counts')
ax.bar(indices, insertion_counts, label='Insertion Counts')

# Adding labels and title
ax.set_xlabel('Compound Reference Index')
ax.set_ylabel('Read Counts')
ax.set_title('Successful Twin Edit Counts - All Alleles')

ax.set_xticks(indices)
ax.set_xticklabels([str(x) for x in range(ins_start, ins_end+1)], rotation=45, ha='center')
# Dynamic? >35 x-tick = font 8 & rotate 45
ax.tick_params(axis='x', labelsize=8)
ax.legend()
plt.tight_layout()


# Display the plot
plt.savefig(fig_root + '.successful_twin_edit_counts_total.pdf')

# Display the plot
plt.show()

In [ ]:
# Indices for the x-axis
indices = np.arange(len(edit_counts))

# Plotting
fig, ax = plt.subplots(figsize=(14, 6)) 

# Stacked bar plot
ax.bar(indices, edit_counts, label='Total TE Edit Counts')
ax.bar(indices, from_right_all_edit_counts, label='From Right All TE Edit Counts')
ax.bar(indices, from_left_all_edit_counts, label='From Left All TE Edit Counts')
ax.bar(indices, perfect_edit_counts, label='Perfect TE Edit Counts')
ax.bar(indices, deletion_counts, label='Deletion Counts')
ax.bar(indices, insertion_counts, label='Insertion Counts')

# Adding labels and title
ax.set_xlabel('Compound Reference Index')
ax.set_ylabel('Read Counts')
ax.set_title('Successful Twin Edit Counts - Edited Alleles')

ax.set_xticks(indices)
ax.set_xticklabels([str(x) for x in range(ins_start, ins_end+1)], rotation=45, ha='center')
# Dynamic? >35 x-tick = font 8 & rotate 45
ax.tick_params(axis='x', labelsize=8)
ax.legend()
plt.tight_layout()


# Display the plot
plt.savefig(fig_root + '.successful_twin_edit_counts.pdf')

# Display the plot
plt.show()

In [ ]:
# Indices for the x-axis
indices = np.arange(len(edit_counts))

# Plotting
fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(14, 18)) 

# Stacked bar plot
ax1.bar(indices, edit_counts, label='Total TE Edit Counts')
ax1.bar(indices, edit_counts_with_deletions, label='Total TE Edit Counts (With indels)')

# Adding labels and title
ax1.set_xlabel('Compound Reference Index')
ax1.set_ylabel('Read Counts')
ax1.set_title('Successful Twin Edit Counts')

ax1.set_xticks(indices)
ax1.set_xticklabels([str(x) for x in range(ins_start, ins_end+1)], rotation=45, ha='center')
ax1.tick_params(axis='x', labelsize=8)
ax1.legend()

# Stacked bar plot
ax2.bar(indices, from_left_all_edit_counts, label='From Left All TE Edit Counts')
ax2.bar(indices, from_left_all_edit_counts_with_deletions, label='From Left All TE Edit Counts (With indels)')

# Adding labels and title
ax2.set_xlabel('Compound Reference Index')
ax2.set_ylabel('Read Counts')
ax2.set_title('Twin Edit From Left Counts')

ax2.set_xticks(indices)
ax2.set_xticklabels([str(x) for x in range(ins_start, ins_end+1)], rotation=45, ha='center')
ax2.tick_params(axis='x', labelsize=8)
ax2.legend()


# Stacked bar plot
ax3.bar(indices, from_right_all_edit_counts, label='From Right All TE Edit Counts')
ax3.bar(indices, from_right_all_edit_counts_with_deletions, label='From Right All TE Edit Counts (With indels)')

# Adding labels and title
ax3.set_xlabel('Compound Reference Index')
ax3.set_ylabel('Read Counts')
ax3.set_title('Twin Edit From Right Counts')

ax3.set_xticks(indices)
ax3.set_xticklabels([str(x) for x in range(ins_start, ins_end+1)], rotation=45, ha='center')
# Dynamic? >35 x-tick = font 8 & rotate 45
ax3.tick_params(axis='x', labelsize=8)
ax3.legend()
plt.tight_layout()


# Display the plot
plt.savefig(fig_root + '.successful_twin_edit_counts_triple.pdf')

# Display the plot
plt.show()


In [22]:
# def add_sgRNA_to_ax(ax,sgRNA_intervals,sgRNA_y_start,sgRNA_y_height,amp_len,x_offset=0,sgRNA_mismatches=None,sgRNA_names=None,sgRNA_rows=None,font_size=None,clip_on=True,label_at_zero=False):
#     """
#     Draw one or more sgRNA annotations on a Matplotlib Axes that shares the x-axis
#     (reference/amplicon coordinates) with a heatmap of alleles.

#     Each sgRNA is represented by a semi-transparent rectangle spanning its start and
#     end coordinates. Optional per-sgRNA mismatches are overlaid as red sub-rectangles,
#     and optional names are placed to the immediate left of each sgRNA. When multiple
#     sgRNAs overlap in x, they can be vertically stacked by assigning them to distinct
#     "rows" to avoid visual overlap.

#     Args:
#         ax (matplotlib.axes.Axes): Target axes to draw sgRNA rectangles and labels.
#         sgRNA_intervals (list[tuple[int,int]]): For each sgRNA, a (start, end) tuple
#             in reference coordinates. Values can extend beyond [0, amp_len-1] and
#             will be clipped to the plotable region. Start and end are inclusive.
#         sgRNA_y_start (float): Baseline y coordinate for the first sgRNA row, in the
#             data coordinate system of ax.
#         sgRNA_y_height (float): Total vertical span available for all sgRNA rows. If
#             multiple rows are used, each row's height is sgRNA_y_height / n_rows.
#         amp_len (int): Length of the amplicon/reference plotted on the x-axis.
#         x_offset (int, optional): Horizontal offset to apply to all sgRNA x positions
#             when the plotted x coordinates are shifted relative to reference indexing.
#             Defaults to 0.
#         sgRNA_mismatches (list[list[int]] | None, optional): For each sgRNA, a list of
#             0-based indices (relative to the sgRNA start) where mismatches should be
#             highlighted in red. Defaults to None.
#         sgRNA_names (list[str] | None, optional): Optional text labels for each sgRNA.
#             If provided, the label is drawn to the left of the sgRNA span. Defaults to None.
#         sgRNA_rows (list[int] | None, optional): Row index for each sgRNA used to
#             vertically stack overlapping sgRNAs. 0 is the top-most row. If None, all
#             sgRNAs are placed on row 0. Defaults to None.
#         font_size (int | str | None, optional): Font size for sgRNA labels. If None,
#             matplotlib.rcParams['font.size'] is used. Defaults to None.
#         clip_on (bool, optional): Whether sgRNA patches/text should be clipped to
#             the Axes' view limits. Set False to allow drawing outside the plot area
#             (useful if labels should appear beyond the axes). Defaults to True.
#         label_at_zero (bool, optional): If True and the left-most sgRNA would be
#             labeled off-plot, anchor the label at x=0 instead. Defaults to False.

#     Notes:
#         - Intervals that extend beyond [0, amp_len-1] are visually capped with small
#           white rectangles to indicate clipping.
#         - The function only adds artists (patches/text) to the provided Axes and
#           does not alter limits, ticks, or other axis properties.
#     """

#     if font_size is None:
#         font_size = matplotlib.rcParams['font.size']

#     # If rows are not provided, draw all sgRNAs on a single row
#     if sgRNA_rows is None:
#         sgRNA_rows = [0]*len(sgRNA_intervals)
#     max_sgRNA_row = max(sgRNA_rows)+1
#     # Height allocated per row
#     this_sgRNA_y_height = sgRNA_y_height/float(max_sgRNA_row)

#     # Track left-most sgRNA to optionally place a generic label
#     min_sgRNA_x = None
#     label_left_sgRNA = True

#     for idx, sgRNA_int in enumerate(sgRNA_intervals):
#         # Clip interval to the drawable region
#         this_sgRNA_start = max(0, sgRNA_int[0])
#         this_sgRNA_end = min(sgRNA_int[1], amp_len - 1)
#         if this_sgRNA_start > amp_len or this_sgRNA_end < 0:
#             # Entire interval falls outside the region; skip
#             continue

#         # Compute the per-row baseline start
#         this_sgRNA_y_start = sgRNA_y_start + this_sgRNA_y_height*sgRNA_rows[idx]

#         # Main sgRNA span as a translucent rectangle
#         ax.add_patch(
#             patches.Rectangle(
#                 (x_offset+this_sgRNA_start, this_sgRNA_y_start),
#                 1+this_sgRNA_end-this_sgRNA_start,
#                 this_sgRNA_y_height,
#                 facecolor=(0, 0, 0, 0.15),
#                 clip_on=clip_on
#             )
#         )

#         # If clipped on the left, draw a small white cap to indicate truncation
#         if (this_sgRNA_start) != sgRNA_int[0]:
#             ax.add_patch(
#                 patches.Rectangle(
#                     (x_offset + 0.1+this_sgRNA_start, this_sgRNA_y_start),
#                     0.1,
#                     this_sgRNA_y_height,
#                     facecolor='w',
#                     clip_on=clip_on
#                 )
#             )
#         # If clipped on the right, draw a white cap
#         if this_sgRNA_end != sgRNA_int[1]:
#             ax.add_patch(
#                 patches.Rectangle(
#                     (x_offset + 0.8+this_sgRNA_end, this_sgRNA_y_start),
#                     0.1,
#                     this_sgRNA_y_height,
#                     facecolor='w',
#                     clip_on=clip_on
#                 )
#             )

#         # Optional per-sgRNA mismatches overlaid in red
#         if sgRNA_mismatches is not None:
#             this_sgRNA_mismatches = sgRNA_mismatches[idx]
#             for mismatch in this_sgRNA_mismatches:
#                 mismatch_plot_pos = sgRNA_int[0] + mismatch
#                 # Only draw if mismatch falls within the drawable region
#                 if mismatch_plot_pos > 0 and mismatch_plot_pos < amp_len - 1:
#                     ax.add_patch(
#                         patches.Rectangle(
#                             (x_offset+ mismatch_plot_pos, this_sgRNA_y_start),
#                             1,
#                             this_sgRNA_y_height,
#                             facecolor='r',
#                             clip_on=clip_on
#                         )
#                     )

#         # Track the minimum (left-most) start for optional generic labeling
#         if not min_sgRNA_x:
#             min_sgRNA_x = this_sgRNA_start
#         if this_sgRNA_start < min_sgRNA_x:
#             min_sgRNA_x = this_sgRNA_start

#         # Optional per-sgRNA label
#         if sgRNA_names is not None and idx < len(sgRNA_names) and sgRNA_names[idx] != "":
#             if (label_at_zero and x_offset + this_sgRNA_start < len(sgRNA_names[idx])*0.66):
#                 ax.text(
#                     0,
#                     this_sgRNA_y_start + this_sgRNA_y_height/2,
#                     sgRNA_names[idx] + " ",
#                     horizontalalignment='left',
#                     verticalalignment='center',
#                     fontsize = font_size
#                 )
#             else:
#                 ax.text(
#                     x_offset+this_sgRNA_start,
#                     this_sgRNA_y_start + this_sgRNA_y_height/2,
#                     sgRNA_names[idx] + " ",
#                     horizontalalignment='right',
#                     verticalalignment='center',
#                     fontsize = font_size
#                 )
#             label_left_sgRNA = False  # At least one sgRNA has been explicitly labeled

#     # If nothing was explicitly labeled, draw a generic label near the left-most sgRNA
#     if min_sgRNA_x is not None and label_left_sgRNA:
#         if (label_at_zero and x_offset + min_sgRNA_x < 5):
#             ax.text(0, this_sgRNA_y_start + this_sgRNA_y_height/2, 'sgRNA ', horizontalalignment='left', verticalalignment='center', fontsize=font_size)
#         else:
#             ax.text(x_offset+min_sgRNA_x, this_sgRNA_y_start + this_sgRNA_y_height/2, 'sgRNA ', horizontalalignment='right', verticalalignment='center', fontsize=font_size)

#### Modified function to allow right-side labels ####
# // ...existing code...
# def add_sgRNA_to_ax(ax,
#                     sgRNA_intervals,
#                     sgRNA_y_start,
#                     sgRNA_y_height,
#                     amp_len,
#                     x_offset=0,
#                     sgRNA_mismatches=None,
#                     sgRNA_names=None,
#                     sgRNA_rows=None,
#                     font_size=None,
#                     clip_on=True,
#                     label_at_zero=False,
#                     sgRNA_label_sides=None):
#     """
#     Draw one or more sgRNA annotations on a Matplotlib Axes.

#     Each sgRNA is a semi-transparent rectangle (start..end). Optional mismatches
#     are red sub-rectangles. Labels can be placed per sgRNA either to the left
#     or right of the rectangle via sgRNA_label_sides.

#     Args:
#         ax (matplotlib.axes.Axes)
#         sgRNA_intervals (list[tuple[int,int]])
#         sgRNA_y_start (float)
#         sgRNA_y_height (float)
#         amp_len (int)
#         x_offset (int)
#         sgRNA_mismatches (list[list[int]] | None)
#         sgRNA_names (list[str] | None)
#         sgRNA_rows (list[int] | None)
#         font_size (int | str | None)
#         clip_on (bool)
#         label_at_zero (bool)
#         sgRNA_label_sides (list[str] | None): Optional per-sgRNA side for label
#             placement. Each entry 'left' or 'right'. If None, all default to
#             'left' (existing behavior). Example: ['left','right'] places the
#             second label to the right of its rectangle.

#     Notes:
#         Right-side labels are anchored just beyond the rectangle end (end+1).
#     """
#     if font_size is None:
#         font_size = matplotlib.rcParams['font.size']

#     if sgRNA_rows is None:
#         sgRNA_rows = [0]*len(sgRNA_intervals)
#     max_sgRNA_row = max(sgRNA_rows)+1
#     this_sgRNA_y_height = sgRNA_y_height/float(max_sgRNA_row)

#     # Normalize label sides
#     if sgRNA_label_sides is None:
#         sgRNA_label_sides = ['left'] * len(sgRNA_intervals)
#     else:
#         # Pad / truncate to length
#         if len(sgRNA_label_sides) < len(sgRNA_intervals):
#             sgRNA_label_sides = sgRNA_label_sides + ['left']*(len(sgRNA_intervals)-len(sgRNA_label_sides))
#         else:
#             sgRNA_label_sides = sgRNA_label_sides[:len(sgRNA_intervals)]

#         # Sanitize values
#         sgRNA_label_sides = [('right' if s.lower()=='right' else 'left') for s in sgRNA_label_sides]

#     min_sgRNA_x = None
#     label_left_sgRNA = True

#     for idx, sgRNA_int in enumerate(sgRNA_intervals):
#         this_sgRNA_start = max(0, sgRNA_int[0])
#         this_sgRNA_end = min(sgRNA_int[1], amp_len - 1)
#         if this_sgRNA_start > amp_len or this_sgRNA_end < 0:
#             continue

#         this_sgRNA_y_row_start = sgRNA_y_start + this_sgRNA_y_height*sgRNA_rows[idx]

#         # Rectangle
#         ax.add_patch(
#             patches.Rectangle(
#                 (x_offset+this_sgRNA_start, this_sgRNA_y_row_start),
#                 1+this_sgRNA_end-this_sgRNA_start,
#                 this_sgRNA_y_height,
#                 facecolor=(0, 0, 0, 0.15),
#                 clip_on=clip_on
#             )
#         )

#         # Clip caps
#         if this_sgRNA_start != sgRNA_int[0]:
#             ax.add_patch(
#                 patches.Rectangle(
#                     (x_offset + 0.1 + this_sgRNA_start, this_sgRNA_y_row_start),
#                     0.1,
#                     this_sgRNA_y_height,
#                     facecolor='w',
#                     clip_on=clip_on
#                 )
#             )
#         if this_sgRNA_end != sgRNA_int[1]:
#             ax.add_patch(
#                 patches.Rectangle(
#                     (x_offset + 0.8 + this_sgRNA_end, this_sgRNA_y_row_start),
#                     0.1,
#                     this_sgRNA_y_height,
#                     facecolor='w',
#                     clip_on=clip_on
#                 )
#             )

#         # Mismatches
#         if sgRNA_mismatches is not None and idx < len(sgRNA_mismatches):
#             for mismatch in sgRNA_mismatches[idx]:
#                 mismatch_plot_pos = sgRNA_int[0] + mismatch
#                 if 0 <= mismatch_plot_pos < amp_len:
#                     ax.add_patch(
#                         patches.Rectangle(
#                             (x_offset + mismatch_plot_pos, this_sgRNA_y_row_start),
#                             1,
#                             this_sgRNA_y_height,
#                             facecolor='r',
#                             clip_on=clip_on
#                         )
#                     )

#         if min_sgRNA_x is None or this_sgRNA_start < min_sgRNA_x:
#             min_sgRNA_x = this_sgRNA_start

#         # Label
#         if sgRNA_names is not None and idx < len(sgRNA_names) and sgRNA_names[idx] != "":
#             side = sgRNA_label_sides[idx]
#             if side == 'left':
#                 # Existing left logic
#                 if (label_at_zero and x_offset + this_sgRNA_start < len(sgRNA_names[idx])*0.66):
#                     ax.text(
#                         0,
#                         this_sgRNA_y_row_start + this_sgRNA_y_height/2,
#                         sgRNA_names[idx] + " ",
#                         horizontalalignment='left',
#                         verticalalignment='center',
#                         fontsize=font_size
#                     )
#                 else:
#                     ax.text(
#                         x_offset + this_sgRNA_start,
#                         this_sgRNA_y_row_start + this_sgRNA_y_height/2,
#                         sgRNA_names[idx] + " ",
#                         horizontalalignment='right',
#                         verticalalignment='center',
#                         fontsize=font_size
#                     )
#             else:  # right
#                 # Place just after rectangle end
#                 label_x = x_offset + this_sgRNA_end + 1.0
#                 ax.text(
#                     label_x,
#                     this_sgRNA_y_row_start + this_sgRNA_y_height/2,
#                     " " + sgRNA_names[idx],
#                     horizontalalignment='left',
#                     verticalalignment='center',
#                     fontsize=font_size
#                 )
#             label_left_sgRNA = False

#     if min_sgRNA_x is not None and label_left_sgRNA:
#         if (label_at_zero and x_offset + min_sgRNA_x < 5):
#             ax.text(0,
#                     this_sgRNA_y_row_start + this_sgRNA_y_height/2,
#                     'sgRNA ',
#                     horizontalalignment='left',
#                     verticalalignment='center',
#                     fontsize=font_size)
#         else:
#             ax.text(x_offset+min_sgRNA_x,
#                     this_sgRNA_y_row_start + this_sgRNA_y_height/2,
#                     'sgRNA ',
#                     horizontalalignment='right',
#                     verticalalignment='center',
#                     fontsize=font_size)
# // ...existing code...
#### End of Modified function to allow right-side labels ####


#### Modified function to allow right-side labels and extend grey pegRNA spacerB span ####
# ...existing code...
def add_sgRNA_to_ax(ax,
                    sgRNA_intervals,
                    sgRNA_y_start,
                    sgRNA_y_height,
                    amp_len,
                    x_offset=0,
                    sgRNA_mismatches=None,
                    sgRNA_names=None,
                    sgRNA_rows=None,
                    font_size=None,
                    clip_on=True,
                    label_at_zero=False,
                    sgRNA_label_sides=None,
                    ref_row_seq=None,                 # WT ref (with '-')
                    extend_left_non_gap=None):         # per-sgRNA left extension in non-gap bases
    """
    Draw one or more sgRNA annotations on a Matplotlib Axes.

    Each sgRNA is a semi-transparent rectangle (start..end). Optional mismatches
    are red sub-rectangles. Labels can be placed per sgRNA either to the left
    or right of the rectangle via sgRNA_label_sides.

    Notes:
        - Right-side labels are anchored just beyond the original rectangle end (end+1).
        - If extend_left_non_gap is set, the visual rectangle start is shifted
          left by that many non-gap bases (skipping '-' in ref_row_seq).
        - If ref_row_seq is provided, the rectangle is split into segments and drawn
          only over non-gap columns (skipping any '-' within the span).
        - Mismatch blocks remain aligned to the original (unextended) sgRNA start.
    """
    if font_size is None:
        font_size = matplotlib.rcParams['font.size']

    if sgRNA_rows is None:
        sgRNA_rows = [0]*len(sgRNA_intervals)
    max_sgRNA_row = max(sgRNA_rows)+1
    this_sgRNA_y_height = sgRNA_y_height/float(max_sgRNA_row)

    # Normalize label sides
    if sgRNA_label_sides is None:
        sgRNA_label_sides = ['left'] * len(sgRNA_intervals)
    else:
        if len(sgRNA_label_sides) < len(sgRNA_intervals):
            sgRNA_label_sides = sgRNA_label_sides + ['left']*(len(sgRNA_intervals)-len(sgRNA_label_sides))
        else:
            sgRNA_label_sides = sgRNA_label_sides[:len(sgRNA_intervals)]
        sgRNA_label_sides = [('right' if s.lower()=='right' else 'left') for s in sgRNA_label_sides]

    # Normalize extension list
    if extend_left_non_gap is None:
        extend_left_non_gap = [0]*len(sgRNA_intervals)
    else:
        if len(extend_left_non_gap) < len(sgRNA_intervals):
            extend_left_non_gap = extend_left_non_gap + [0]*(len(sgRNA_intervals)-len(extend_left_non_gap))
        else:
            extend_left_non_gap = extend_left_non_gap[:len(sgRNA_intervals)]

    def _left_shift_by_non_gaps(row_seq, start_idx, n_non_gaps):
        """Return how many columns to shift left to include n_non_gaps non-'-' bases."""
        if row_seq is None or n_non_gaps <= 0 or start_idx <= 0:
            return 0
        count = 0
        steps = 0
        i = int(start_idx) - 1
        while i >= 0 and count < n_non_gaps:
            if row_seq[i] != '-':
                count += 1
            steps += 1
            i -= 1
        return steps

    def _non_gap_runs(row_seq, start_idx, end_idx):
        """Return list of (run_start, run_end) contiguous non-gap segments in [start_idx, end_idx]."""
        if row_seq is None:
            return [(int(start_idx), int(end_idx))]
        if len(row_seq) == 0:
            return []
        s = max(0, int(start_idx))
        e = min(int(end_idx), len(row_seq) - 1)
        runs = []
        i = s
        while i <= e:
            while i <= e and row_seq[i] == '-':
                i += 1
            if i > e:
                break
            run_start = i
            while i <= e and row_seq[i] != '-':
                i += 1
            run_end = i - 1
            runs.append((run_start, run_end))
        return runs

    min_sgRNA_x = None
    label_left_sgRNA = True

    for idx, sgRNA_int in enumerate(sgRNA_intervals):
        # Original clipped interval (for mismatch alignment and caps)
        this_sgRNA_start = max(0, sgRNA_int[0])
        this_sgRNA_end   = min(sgRNA_int[1], amp_len - 1)
        if this_sgRNA_start > amp_len or this_sgRNA_end < 0:
            continue

        this_sgRNA_y_row_start = sgRNA_y_start + this_sgRNA_y_height*sgRNA_rows[idx]

        # Visual start extended left by N non-gap bases (skip '-')
        left_extra_cols = _left_shift_by_non_gaps(ref_row_seq, this_sgRNA_start, extend_left_non_gap[idx])
        display_start = max(0, this_sgRNA_start - left_extra_cols)

        # Draw as multiple rectangles over non-gap runs only
        runs = _non_gap_runs(ref_row_seq, display_start, this_sgRNA_end)
        for seg_start, seg_end in runs:
            if seg_start > seg_end:
                continue
            ax.add_patch(
                patches.Rectangle(
                    (x_offset + seg_start, this_sgRNA_y_row_start),
                    1 + seg_end - seg_start,
                    this_sgRNA_y_height,
                    facecolor=(0, 0, 0, 0.15),
                    clip_on=clip_on
                )
            )

        # Clip caps (based on original interval vs clipping)
        if this_sgRNA_start != sgRNA_int[0]:
            ax.add_patch(
                patches.Rectangle(
                    (x_offset + 0.1 + this_sgRNA_start, this_sgRNA_y_row_start),
                    0.1,
                    this_sgRNA_y_height,
                    facecolor='w',
                    clip_on=clip_on
                )
            )
        if this_sgRNA_end != sgRNA_int[1]:
            ax.add_patch(
                patches.Rectangle(
                    (x_offset + 0.8 + this_sgRNA_end, this_sgRNA_y_row_start),
                    0.1,
                    this_sgRNA_y_height,
                    facecolor='w',
                    clip_on=clip_on
                )
            )

        # Mismatches (relative to original sgRNA start)
        if sgRNA_mismatches is not None and idx < len(sgRNA_mismatches):
            for mismatch in sgRNA_mismatches[idx]:
                mismatch_plot_pos = sgRNA_int[0] + mismatch
                if 0 <= mismatch_plot_pos < amp_len:
                    ax.add_patch(
                        patches.Rectangle(
                            (x_offset + mismatch_plot_pos, this_sgRNA_y_row_start),
                            1,
                            this_sgRNA_y_height,
                            facecolor='r',
                            clip_on=clip_on
                        )
                    )

        # For left-anchored label and min_x heuristic, use first visible segment start
        leftmost_visible = runs[0][0] if runs else display_start
        if min_sgRNA_x is None or leftmost_visible < min_sgRNA_x:
            min_sgRNA_x = leftmost_visible

        # Label
        if sgRNA_names is not None and idx < len(sgRNA_names) and sgRNA_names[idx] != "":
            side = sgRNA_label_sides[idx]
            if side == 'left':
                anchor_x = x_offset + leftmost_visible
                if (label_at_zero and anchor_x < len(sgRNA_names[idx])*0.66):
                    ax.text(
                        0,
                        this_sgRNA_y_row_start + this_sgRNA_y_height/2,
                        sgRNA_names[idx] + " ",
                        horizontalalignment='left',
                        verticalalignment='center',
                        fontsize=font_size
                    )
                else:
                    ax.text(
                        anchor_x,
                        this_sgRNA_y_row_start + this_sgRNA_y_height/2,
                        sgRNA_names[idx] + " ",
                        horizontalalignment='right',
                        verticalalignment='center',
                        fontsize=font_size
                    )
            else:  # right
                label_x = x_offset + this_sgRNA_end + 1.0
                ax.text(
                    label_x,
                    this_sgRNA_y_row_start + this_sgRNA_y_height/2,
                    " " + sgRNA_names[idx],
                    horizontalalignment='left',
                    verticalalignment='center',
                    fontsize=font_size
                )
            label_left_sgRNA = False

    if min_sgRNA_x is not None and label_left_sgRNA:
        if (label_at_zero and x_offset + min_sgRNA_x < 5):
            ax.text(0,
                    this_sgRNA_y_row_start + this_sgRNA_y_height/2,
                    'sgRNA ',
                    horizontalalignment='left',
                    verticalalignment='center',
                    fontsize=font_size)
        else:
            ax.text(x_offset+min_sgRNA_x,
                    this_sgRNA_y_row_start + this_sgRNA_y_height/2,
                    'sgRNA ',
                    horizontalalignment='right',
                    verticalalignment='center',
                    fontsize=font_size)
# ...existing code...
#### End of Modified function to allow right-side labels and extend grey pegRNA spacerB span ####


def get_nuc_color(nuc, alpha):
    """
    Return a consistent RGBA color tuple for a nucleotide or special token.

    Args:
        nuc (str): One of {'A','T','C','G','N','INS','DEL','-'} or any other string.
            'N' denotes ambiguous; '-' denotes gap. 'INS'/'DEL' share the same color to
            visually group indels. Any unknown token results in a deterministic color
            derived from its character codes.
        alpha (float): Alpha channel in [0.0, 1.0] controlling transparency.

    Returns:
        tuple: (r, g, b, a) with values in [0.0, 1.0].
    """
    get_color=lambda x, y, z: (x/255.0, y/255.0, z/255.0, alpha)
    if nuc == "A":
        return get_color(127, 201, 127)
    elif nuc == "T":
        return get_color(190, 174, 212)
    elif nuc == "C":
        return get_color(253, 192, 134)
    elif nuc == "G":
        return get_color(255, 255, 153)
    elif nuc == "N":
        return get_color(200, 200, 200)
    elif nuc == "INS":
#        return get_color(185,219,253)
#        return get_color(177,125,76)
        return get_color(193, 129, 114)
    elif nuc == "DEL":
        #return get_color(177,125,76)
#        return get_color(202,109,87)
        return get_color(193, 129, 114)
    elif nuc == "-":
        #return get_color(177,125,76)
#        return get_color(202,109,87)
        return get_color(30, 30, 30)
    elif nuc == " ":
        # white space for padding
        return get_color(255, 255, 255)
    else: #return a random color (that is based on the nucleotide given)
        charSum = 0
        for char in nuc.upper():
            thisval = ord(char) - 65 #'A' is 65
            if thisval < 0 or thisval > 90:
                thisval = 0
            charSum += thisval
        charSum = (charSum/len(nuc))/90.0

        return (charSum, (1-charSum), (2*charSum*(1-charSum)))
    

### Allele plot
# We need to customize the seaborn heatmap class and function
class Custom_HeatMapper(sns.matrix._HeatMapper):
    """
    Extension of seaborn's private _HeatMapper to support per-element annotation style
    (per-element text properties) and to suppress the colorbar.

    This utility mirrors seaborn.heatmap internals while allowing an "annot" matrix to be
    styled cell-by-cell via a parallel matrix of dictionaries (per_element_annot_kws), where
    each dictionary can specify matplotlib.text.Text properties for the corresponding cell.

    Caution: sns.matrix._HeatMapper is a private API and may change across seaborn versions.
    """

    def __init__(self, data, vmin, vmax, cmap, center, robust, annot, fmt,
                 annot_kws,per_element_annot_kws,cbar, cbar_kws,
                 xticklabels=True, yticklabels=True, mask=None):
        """
        Initialize the heatmap plotter and capture optional per-element annotation styles.

        Args mirror seaborn.heatmap/_HeatMapper with the following addition:
            per_element_annot_kws (np.ndarray | list | None): Same shape as `annot` where each
                element is a dict of Text properties applied to that cell's annotation.
                If None, an empty dict is used for every cell.
        """
        super(Custom_HeatMapper, self).__init__(data, vmin, vmax, cmap, center, robust, annot, fmt,
                 annot_kws, cbar, cbar_kws,
                 xticklabels, yticklabels, mask)

        # Prepare a mirror structure for per-element annotation keyword arguments
        if annot is not None:
            if per_element_annot_kws is None:
                self.per_element_annot_kws=np.empty_like(annot, dtype=object)
                self.per_element_annot_kws[:]=dict()
            else:
                self.per_element_annot_kws=per_element_annot_kws

    # add per element dict to style the annotation
    def _annotate_heatmap(self, ax, mesh):
        """Add textual labels with the value in each cell.

        This override allows passing a per-cell dictionary of Text properties to fine-tune
        the appearance (e.g., bold substitutions) while preserving seaborn's luminance-based
        foreground color choice.
        """
        mesh.update_scalarmappable()
        xpos, ypos = np.meshgrid(ax.get_xticks(), ax.get_yticks())

        # Iterate the mesh values, facecolors, annotations, and per-cell styles in lock-step
        for x, y, m, color, val, per_element_dict  in zip(xpos.flat, ypos.flat,
                                       mesh.get_array().flat, mesh.get_facecolors(),
                                       self.annot_data.flat, self.per_element_annot_kws.flat):
            #print per_element_dict
            if m is not np.ma.masked:
                l = sns.utils.relative_luminance(color)
                text_color = ".15" if l > .408 else "w"
                annotation = ("{:" + self.fmt + "}").format(str(val))
                text_kwargs = dict(color=text_color, ha="center", va="center")
                text_kwargs.update(self.annot_kws)
                text_kwargs.update(per_element_dict)

                ax.text(x, y, annotation, **text_kwargs)


    # removed the colorbar
    def plot(self, ax, cax, kws):
        """Draw the heatmap body and tick labels on the provided Axes.

        This version deliberately avoids attaching a colorbar and leaves any colorbar
        management to the caller.
        """
        # Remove all the Axes spines for a cleaner matrix look
        sns.utils.despine(ax=ax, left=True, bottom=True)

        # Draw the heatmap as a pcolormesh for performance on large matrices
        # If a Normalize is supplied, do NOT also pass vmin/vmax.
        if 'norm' in kws and kws['norm'] is not None:
            mesh = ax.pcolormesh(self.plot_data, cmap=self.cmap, **kws)
        else:
            mesh = ax.pcolormesh(self.plot_data, vmin=self.vmin, vmax=self.vmax,
                                 cmap=self.cmap, **kws)

        # Set axis limits to span the matrix exactly
        ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))

        # Add row and column labels
        ax.set(xticks=self.xticks, yticks=self.yticks)
        xtl = ax.set_xticklabels(self.xticklabels)
        ytl = ax.set_yticklabels(self.yticklabels, rotation="vertical", va='center')

        # Possibly rotate them if they overlap after layout
        plt.draw()
        if sns.utils.axis_ticklabels_overlap(xtl):
            plt.setp(xtl, rotation="vertical")
        if sns.utils.axis_ticklabels_overlap(ytl):
            plt.setp(ytl, rotation="horizontal")

        # Add the axis labels
        ax.set(xlabel=self.xlabel, ylabel=self.ylabel)

        # Annotate the cells with the formatted values
        if self.annot:
            self._annotate_heatmap(ax, mesh)


def custom_heatmap(data, vmin=None, vmax=None, cmap=None, center=None, robust=False,
            annot=None, fmt=".2g", annot_kws=None,per_element_annot_kws=None,
            linewidths=0, linecolor="white",
            cbar=True, cbar_kws=None, cbar_ax=None,
            square=False, ax=None, xticklabels=True, yticklabels=True,
            mask=None,
            **kwargs):
    """
    Convenience wrapper around Custom_HeatMapper to draw a heatmap matrix with optional
    per-element annotation styling and without a colorbar by default.

    Args:
        data (np.ndarray | list): 2D array of numeric values to visualize.
        vmin, vmax (float | None): Colormap scaling bounds.
        cmap (matplotlib.colors.Colormap | str | None): Colormap to use.
        center (float | None): If set, shift the colormap center to this value.
        robust (bool): If True, use robust quantiles rather than min/max for colormap scaling.
        annot (np.ndarray | list | None): 2D array of values/strings to annotate each cell.
        fmt (str): Format string applied to annotations.
        annot_kws (dict | None): Global matplotlib.text.Text properties applied to all annotations.
        per_element_annot_kws (np.ndarray | list | None): Same shape as annot; each element is a
            dict of Text properties applied to that cell, allowing per-cell styles.
        linewidths (float): Line width between cells (pcolormesh edge widths).
        linecolor (str): Line color between cells.
        cbar (bool): Present for API parity; colorbar is not created by this function.
        cbar_kws (dict | None): Ignored here; reserved for compatibility.
        cbar_ax (matplotlib.axes.Axes | None): Ignored here; reserved for compatibility.
        square (bool): If True, set aspect to equal so each cell is square.
        ax (matplotlib.axes.Axes | None): Axes to draw into; if None, uses current axes.
        xticklabels, yticklabels: Tick label configuration as in seaborn.heatmap.
        mask (np.ndarray | None): Boolean mask specifying cells not to plot.
        **kwargs: Additional arguments passed to Axes.pcolormesh (e.g., shading, antialiased).

    Returns:
        matplotlib.axes.Axes: The Axes containing the heatmap.
    """

    # Initialize the plotter object
    plotter = Custom_HeatMapper(data, vmin, vmax, cmap, center, robust, annot, fmt,
                          annot_kws, per_element_annot_kws, cbar, cbar_kws, xticklabels,
                          yticklabels, mask)


    # Add the pcolormesh kwargs here
    kwargs["linewidths"] = linewidths
    kwargs["edgecolor"] = linecolor

    # Draw the plot and return the Axes
    if ax is None:
        ax = plt.gca()
    if square:
        ax.set_aspect("equal")
    plotter.plot(ax, cbar_ax, kwargs)
    return ax


def get_rows_for_sgRNA_annotation(sgRNA_intervals, amp_len):
    """
    Assign a vertical "row" for each sgRNA interval so that overlapping intervals
    are staggered and do not visually collide when drawn.

    The algorithm greedily places each interval on the top-most row that does not
    already contain any of its covered x positions. Occupancy is tracked per integer
    x position between the interval's (clipped) start and end.

    Args:
        sgRNA_intervals (list[tuple[int,int]]): List of (start, end) sgRNA spans in reference
            coordinates. Intervals are clipped to [0, amp_len-1] for overlap calculations.
        amp_len (int): Amplicon/reference length for clipping.

    Returns:
        np.ndarray: Row indices (int) per sgRNA, where 0 is the top-most row. The rows are
        inverted (highest row index becomes 0) so that earlier rows appear visually higher
        when drawn relative to negative y offsets.
    """
    # figure out how many rows are needed to show all sgRNAs
    sgRNA_plot_rows = [0]*len(sgRNA_intervals) # which row each sgRNA should be plotted on
    sgRNA_plot_occupancy = [] # which idxs are already filled on each row
    sgRNA_plot_occupancy.append([])
    for idx, sgRNA_int in enumerate(sgRNA_intervals):
        this_sgRNA_start = max(0, sgRNA_int[0])
        this_sgRNA_end = min(sgRNA_int[1], amp_len - 1)
        curr_row = 0
        if this_sgRNA_start > amp_len or this_sgRNA_end < 0:
            # Interval entirely outside; place on row 0 and continue
            sgRNA_plot_rows[idx] = curr_row
            continue
        # Bump the row until there is no position overlap with already-placed intervals
        while len(np.intersect1d(sgRNA_plot_occupancy[curr_row], range(this_sgRNA_start, this_sgRNA_end))) > 0:
            next_row = curr_row + 1
            if not next_row in sgRNA_plot_occupancy:
                sgRNA_plot_occupancy.append([])
            curr_row = next_row
        sgRNA_plot_rows[idx] = curr_row
        # Mark occupancy for the chosen row
        sgRNA_plot_occupancy[curr_row].extend(range(this_sgRNA_start, this_sgRNA_end))
    # Invert rows so that the last-created (lowest) row is drawn lowest when using negative offsets
    return(np.subtract(max(sgRNA_plot_rows), sgRNA_plot_rows))


# def prep_alleles_table(df_alleles, reference_seq, MAX_N_ROWS, MIN_FREQUENCY):
#     """
#     Prepare matrices and metadata required to render an allele heatmap.

#     This function converts a subset of rows from a CRISPResso allele table into:
#       - X: numeric matrix encoding per-position allele bases for each displayed allele
#       - annot: parallel matrix of string characters for on-cell annotation
#       - y_labels: formatted labels per allele row (percent and read count)
#       - insertion_dict: mapping of row index to a list of (start,end) x-spans that denote
#         insertion events (identified by runs of '-' in the reference alignment)
#       - per_element_annot_kws: per-cell Text style dictionaries (e.g., bold substitutions)
#       - is_reference: boolean flags indicating whether the allele is identical to the
#         provided reference sequence without indels

#     Selection is limited to the top MAX_N_ROWS rows that meet MIN_FREQUENCY based on
#     df_alleles['%Reads'].

#     Args:
#         df_alleles (pd.DataFrame): Allele table indexed by Aligned_Sequence (or similar),
#             containing at least columns: 'Reference_Sequence', '#Reads', '%Reads'.
#         reference_seq (str): Ungapped reference sequence used to determine reference rows.
#         MAX_N_ROWS (int): Maximum number of allele rows to include.
#         MIN_FREQUENCY (float): Minimum percentage (df_alleles['%Reads']) to include a row.

#     Returns:
#         tuple:
#             X (list[list[int]]): Numeric-encoded alleles using dna_to_numbers mapping.
#             annot (list[list[str]]): Same shape as X with literal characters per cell.
#             y_labels (list[str]): Display labels per row (e.g., "12.34% (123 reads)").
#             insertion_dict (defaultdict(list)): row_index -> list of (start, end) insertion spans.
#             per_element_annot_kws (np.ndarray): Per-cell dicts of Text properties used for styling.
#             is_reference (list[bool]): True if row exactly matches reference_seq and has no indels.
#     """
#     dna_to_numbers={'-':0,'A':1,'T':2,'C':3,'G':4,'N':5}
#     seq_to_numbers= lambda seq: [dna_to_numbers[x] for x in seq]
#     X=[]
#     annot=[]
#     y_labels=[]
#     insertion_dict=defaultdict(list)
#     per_element_annot_kws=[]
#     is_reference=[]

#     # Regex to find contiguous gap runs ('-') in the reference alignment; these correspond to
#     # insertions in the read relative to the reference.
#     re_find_indels=re.compile("(-*-)")
#     idx_row=0
#     for idx, row in df_alleles[df_alleles['%Reads']>=MIN_FREQUENCY][:MAX_N_ROWS].iterrows():
#         # Encode the allele (index) sequence
#         X.append(seq_to_numbers(idx.upper()))
#         annot.append(list(idx))

#         # Track insertion spans based on gaps in the reference sequence
#         has_indels = False
#         for p in re_find_indels.finditer(row['Reference_Sequence']):
#             has_indels = True
#             insertion_dict[idx_row].append((p.start(), p.end()))

#         # Build y-axis labels with percentage and read count
#         y_labels.append('%.2f%% (%d reads)' % (row['%Reads'], row['#Reads']))
#         if idx == reference_seq and not has_indels:
#             is_reference.append(True)
#         else:
#             is_reference.append(False)

#         idx_row+=1

#         # Detect substitutions (non-gap mismatches) to style them in bold/black
#         idxs_sub= [i_sub for i_sub in range(len(idx)) if \
#                    (row['Reference_Sequence'][i_sub]!=idx[i_sub]) and \
#                    (row['Reference_Sequence'][i_sub]!='-') and\
#                    (idx[i_sub]!='-')]
#         to_append=np.array([{}]*len(idx), dtype=object)
#         to_append[ idxs_sub]={'weight':'bold', 'color':'black','size':16}
#         per_element_annot_kws.append(to_append)

#     return X, annot, y_labels, insertion_dict, per_element_annot_kws, is_reference


def prep_alleles_table(df_alleles, reference_seq, ref_aln_seq_region, twin_aln_seq_region, MAX_N_ROWS, MIN_FREQUENCY):
    """
    Prepare matrices and metadata required to render an allele heatmap.

    This function converts a subset of rows from a CRISPResso allele table into:
      - X: numeric matrix encoding per-position allele bases for each displayed allele
      - annot: parallel matrix of string characters for on-cell annotation
      - y_labels: formatted labels per allele row (percent and read count)
      - insertion_dict: mapping of row index to a list of (start,end) x-spans that denote
        insertion events (identified by runs of '-' in the reference alignment)
      - per_element_annot_kws: per-cell Text style dictionaries (e.g., bold substitutions)
      - is_reference: boolean flags indicating whether the allele is identical to the
        provided reference sequence without indels

    Selection is limited to the top MAX_N_ROWS rows that meet MIN_FREQUENCY based on
    df_alleles['%Reads'].

    Args:
        df_alleles (pd.DataFrame): Allele table indexed by Aligned_Sequence (or similar),
            containing at least columns: 'Reference_Sequence', '#Reads', '%Reads'.
        reference_seq (str): Ungapped reference sequence used to determine reference rows.
        MAX_N_ROWS (int): Maximum number of allele rows to include.
        MIN_FREQUENCY (float): Minimum percentage (df_alleles['%Reads']) to include a row.

    Returns:
        tuple:
            X (list[list[int]]): Numeric-encoded alleles using dna_to_numbers mapping.
            annot (list[list[str]]): Same shape as X with literal characters per cell.
            y_labels (list[str]): Display labels per row (e.g., "12.34% (123 reads)").
            insertion_dict (defaultdict(list)): row_index -> list of (start, end) insertion spans.
            per_element_annot_kws (np.ndarray): Per-cell dicts of Text properties used for styling.
            is_reference (list[bool]): True if row exactly matches reference_seq and has no indels.
    """
    dna_to_numbers={'-':0,'A':1,'T':2,'C':3,'G':4,'N':5, ' ':6}
    seq_to_numbers= lambda seq: [dna_to_numbers[x] for x in seq]
    X=[]
    annot=[]
    y_labels=[]
    insertion_dict=defaultdict(list)
    per_element_annot_kws=[]
    is_reference=[]
    num_blanks = 2

    # Regex to find contiguous gap runs ('-') in the reference alignment; these correspond to
    # insertions in the read relative to the reference.
    re_find_indels=re.compile("(-*-)")
    idx_row=0
    for idx, row in df_alleles[df_alleles['%Reads']>=MIN_FREQUENCY][:MAX_N_ROWS].iterrows():
        # Encode the allele (index) sequence
        X.append(seq_to_numbers(idx.upper()))
        annot.append(list(idx))

        # Track insertion spans based on gaps in the reference sequence
        has_indels = False
        for p in re_find_indels.finditer(row['Reference_Sequence']):
            has_indels = True
            insertion_dict[idx_row].append((p.start(), p.end()))

        # Build y-axis labels with percentage and read count
        y_labels.append('%.2f%% (%d reads)' % (row['%Reads'], row['#Reads']))
        if idx == reference_seq and not has_indels:
            is_reference.append(True)
        else:
            is_reference.append(False)

        idx_row+=1

        # Detect substitutions (non-gap mismatches) to style them in bold/black
        idxs_sub= [i_sub for i_sub in range(len(idx)) if \
                   (row['Reference_Sequence'][i_sub]!=idx[i_sub]) and \
                   (row['Reference_Sequence'][i_sub]!='-') and\
                   (idx[i_sub]!='-')]
        to_append=np.array([{}]*len(idx), dtype=object)
        to_append[ idxs_sub]={'weight':'bold', 'color':'black','size':16}
        per_element_annot_kws.append(to_append)

    if twin_aln_seq_region is not None and len(twin_aln_seq_region) > 0:
        # Append two blank rows to the end of outputs
        for _ in range(num_blanks):
            blank_row = ' ' * len(twin_aln_seq_region)
            X.append(seq_to_numbers(blank_row))
            annot.append(list(blank_row))
            y_labels.append('')
            is_reference.append(False)
            to_append=np.array([{'color':'white'}]*len(blank_row), dtype=object)
            per_element_annot_kws.append(to_append)

        # Append twin_aln_seq_region to the end of outputs
        X.append(seq_to_numbers(twin_aln_seq_region.upper()))
        annot.append(list(twin_aln_seq_region))
        y_labels.append('TwinPE Reference')
        is_reference.append(False)
        # detect substitutions (non-gap mismatches) between twin_aln_seq_region and ref_aln_seq_region to style them in bold/black
        idxs_sub= [i_sub for i_sub in range(len(twin_aln_seq_region)) if \
                   (ref_aln_seq_region[i_sub]!=twin_aln_seq_region[i_sub]) and \
                   (ref_aln_seq_region[i_sub]!='-') and\
                   (twin_aln_seq_region[i_sub]!='-')]
        to_append=np.array([{}]*len(twin_aln_seq_region), dtype=object)
        to_append[ idxs_sub]={'weight':'bold', 'color':'black','size':16}
        per_element_annot_kws.append(to_append)
        # track insertions in twin_aln_seq_region relative to ref_aln_seq_region
        for p in re_find_indels.finditer(ref_aln_seq_region):
            insertion_dict[idx_row+num_blanks].append((p.start(), p.end()))

    return X, annot, y_labels, insertion_dict, per_element_annot_kws, is_reference


def plot_alleles_heatmap(
        reference_seq,
        # twin_aln_seq=twin_aln_seq_region,
        X,
        annot,
        y_labels,
        insertion_dict,
        per_element_annot_kws,
        fig_filename_root=None,
        custom_colors=None,
        SAVE_ALSO_PNG=False,
        plot_cut_point=True,
        cut_point_ind=None,
        sgRNA_intervals=None,
        sgRNA_names=None,
        sgRNA_mismatches=None,
        extend_left_non_gap=None,
        category=None,
        **kwargs):
    """
    Plots alleles in a heatmap (nucleotides color-coded for easy visualization)
    input:
    -reference_seq: sequence of reference allele to plot
    -X: list of numbers representing nucleotides of the allele
    -annot: list of nucleotides (letters) of the allele
    -y_labels: list of labels for each row/allele
    -insertion_dict: locations of insertions -- red squares will be drawn around these
    -per_element_annot_kws: annotations for each cell (e.g. bold for substitutions, etc.)
    -fig_filename_root: figure filename to plot (not including '.pdf' or '.png'). If None, plots are shown interactively.
    -custom_colors: dict of colors to plot (e.g. colors['A'] = (1,0,0,0.4) # red,blue,green,alpha )
    -SAVE_ALSO_PNG: whether to write png file as well
    -plot_cut_point: if false, won't draw 'predicted cleavage' line
    -cut_point_ind: index of cut point (if None, will be plot in the middle calculated as len(reference_seq)/2)
    -sgRNA_intervals: locations where sgRNA is located
    -sgRNA_mismatches: array (for each sgRNA_interval) of locations in sgRNA where there are mismatches
    -sgRNA_names: array (for each sgRNA_interval) of names of sgRNAs (otherwise empty)
    -custom_colors: dict of colors to plot (e.g. colors['A'] = (1,0,0,0.4) # red,blue,green,alpha )
    """
    plot_nuc_len=len(reference_seq)

    # make a color map of fixed colors
    alpha=0.4
    A_color=get_nuc_color('A', alpha)
    T_color=get_nuc_color('T', alpha)
    C_color=get_nuc_color('C', alpha)
    G_color=get_nuc_color('G', alpha)
    INDEL_color = get_nuc_color('N', alpha)
    blank_color = get_nuc_color(' ', alpha)

    if custom_colors is not None:
        hex_alpha = '66'  # this is equivalent to 40% in hexadecimal
        if 'A' in custom_colors:
            A_color = custom_colors['A'] + hex_alpha
        if 'T' in custom_colors:
            T_color = custom_colors['T'] + hex_alpha
        if 'C' in custom_colors:
            C_color = custom_colors['C'] + hex_alpha
        if 'G' in custom_colors:
            G_color = custom_colors['G'] + hex_alpha
        if 'N' in custom_colors:
            INDEL_color = custom_colors['N'] + hex_alpha
        if ' ' in custom_colors:
            blank_color = custom_colors[' '] + hex_alpha

    dna_to_numbers={'-':0,'A':1,'T':2,'C':3,'G':4,'N':5, ' ':6}
    seq_to_numbers= lambda seq: [dna_to_numbers[x] for x in seq]

    # cmap = colors_mpl.ListedColormap([INDEL_color, A_color, T_color, C_color, G_color, INDEL_color, blank_color])
    # cmap.set_over(blank_color)
    # norm = colors_mpl.Normalize(vmin=0, vmax=5)

    # New: 7-color colormap + discrete norm for 0..6
    cmap = colors_mpl.ListedColormap([INDEL_color, A_color, T_color, C_color, G_color, INDEL_color, blank_color])
    bnorm = colors_mpl.BoundaryNorm(np.arange(-0.5, 7.5, 1), cmap.N, clip=False)

    #ref_seq_around_cut=reference_seq[max(0,cut_point-plot_nuc_len/2+1):min(len(reference_seq),cut_point+plot_nuc_len/2+1)]

#    print('per element anoot kws: ' + per_element_annot_kws)
    if len(per_element_annot_kws) > 1:
        per_element_annot_kws=np.vstack(per_element_annot_kws[::-1])
    else:
        per_element_annot_kws=np.array(per_element_annot_kws)
    ref_seq_hm=np.expand_dims(seq_to_numbers(reference_seq), 1).T
    ref_seq_annot_hm=np.expand_dims(list(reference_seq), 1).T

    annot=annot[::-1]
    X=X[::-1]

    N_ROWS=len(X)
    N_COLUMNS=plot_nuc_len

    if N_ROWS < 1:
        fig, ax = plt.subplots()
        fig.text(0.5, 0.5, 'No Alleles', horizontalalignment='center', verticalalignment='center', transform = ax.transAxes)
        ax.set_clip_on(False)

        if fig_filename_root is None:
            plt.show()
        else:
            fig.savefig(fig_filename_root+'.pdf', bbox_inches='tight')
            if SAVE_ALSO_PNG:
                fig.savefig(fig_filename_root+'.png', bbox_inches='tight')
        plt.close(fig)
        return

    sgRNA_rows = []
    num_sgRNA_rows = 0

    if sgRNA_intervals and len(sgRNA_intervals) > 0:
        sgRNA_rows = get_rows_for_sgRNA_annotation(sgRNA_intervals, plot_nuc_len)
        num_sgRNA_rows = max(sgRNA_rows) + 1
        fig=plt.figure(figsize=(plot_nuc_len*0.3, (N_ROWS+1 + num_sgRNA_rows)*0.6))
        gs1 = gridspec.GridSpec(N_ROWS+2, N_COLUMNS)
        gs2 = gridspec.GridSpec(N_ROWS+2, N_COLUMNS)
        #ax_hm_ref heatmap for the reference
        ax_hm_ref=plt.subplot(gs1[0:1,:])
        ax_hm=plt.subplot(gs2[2:,:])
    else:
        fig=plt.figure(figsize=(plot_nuc_len*0.3, (N_ROWS+1)*0.6))
        gs1 = gridspec.GridSpec(N_ROWS+1, N_COLUMNS)
        gs2 = gridspec.GridSpec(N_ROWS+1, N_COLUMNS)
        #ax_hm_ref heatmap for the reference
        ax_hm_ref=plt.subplot(gs1[0,:])
        ax_hm=plt.subplot(gs2[1:,:])


    custom_heatmap(ref_seq_hm, annot=ref_seq_annot_hm, annot_kws={'size':16}, cmap=cmap, fmt='s', ax=ax_hm_ref, norm=bnorm, vmin=None, vmax=None, square=True)
    custom_heatmap(X, annot=np.array(annot), annot_kws={'size':16}, cmap=cmap, fmt='s', ax=ax_hm, norm=bnorm, vmin=None, vmax=None, square=True, per_element_annot_kws=per_element_annot_kws)
    

    # place ticks at row centers
    ax_hm.yaxis.tick_right()
    ax_hm.set_yticks(np.arange(N_ROWS) + 0.5)
    # apply labels in reverse order
    ax_hm.set_yticklabels(y_labels[::-1], rotation=True, va='center')
    # turn off only the 2nd and 3rd tick marks (Python is 0-based)
    for idx in [1, 2]:
        tick = ax_hm.yaxis.get_major_ticks()[idx]
        tick.tick1line.set_visible(False)  # left tick
        tick.tick2line.set_visible(False)  # right tick
    ax_hm.xaxis.set_ticks([])

    #### Orginal method - includes right-side labels####
    # if sgRNA_intervals and len(sgRNA_intervals) > 0:
    #     this_sgRNA_y_start = -1*num_sgRNA_rows
    #     this_sgRNA_y_height = num_sgRNA_rows - 0.3
    #     # add_sgRNA_to_ax(ax_hm_ref, sgRNA_intervals, sgRNA_y_start=this_sgRNA_y_start, sgRNA_y_height=this_sgRNA_y_height, amp_len=plot_nuc_len, font_size='small', clip_on=False, sgRNA_names=sgRNA_names, sgRNA_mismatches=sgRNA_mismatches, x_offset=0, label_at_zero=True, sgRNA_rows=sgRNA_rows)
    #     add_sgRNA_to_ax(ax_hm_ref, sgRNA_intervals, sgRNA_y_start=this_sgRNA_y_start, sgRNA_y_height=this_sgRNA_y_height, amp_len=plot_nuc_len, font_size='small', clip_on=False, sgRNA_names=sgRNA_names, sgRNA_mismatches=sgRNA_mismatches, x_offset=0, label_at_zero=True, sgRNA_rows=sgRNA_rows, sgRNA_label_sides=['left','right'])
    #### Orginal method ####

    #### New method - includes right-side labels and extended grey pegRNA spacerB span ####
    # if sgRNA_intervals and len(sgRNA_intervals) > 0:
    #     this_sgRNA_y_start = -1*num_sgRNA_rows
    #     this_sgRNA_y_height = num_sgRNA_rows - 0.3

    #     # Extend only the second sgRNA by 3 non-gap bases to the left in WT reference row
    #     extend_left = [0]*len(sgRNA_intervals)
    #     if len(extend_left) > 1:
    #         extend_left[1] = 3

    #     add_sgRNA_to_ax(
    #         ax_hm_ref,
    #         sgRNA_intervals,
    #         sgRNA_y_start=this_sgRNA_y_start,
    #         sgRNA_y_height=this_sgRNA_y_height,
    #         amp_len=plot_nuc_len,
    #         font_size='small',
    #         clip_on=False,
    #         sgRNA_names=sgRNA_names,
    #         sgRNA_mismatches=sgRNA_mismatches,
    #         x_offset=0,
    #         label_at_zero=True,
    #         sgRNA_rows=sgRNA_rows,
    #         sgRNA_label_sides=['left','right'],
    #         ref_row_seq=ref_aln_seq_region,         # WT ref alignment row (with '-')
    #         extend_left_non_gap=extend_left         # apply extension
    #     )
    #### End of New method ####

    #### Newer method - includes right-side labels and extended grey pegRNA spacerB span by caller ####
    if sgRNA_intervals and len(sgRNA_intervals) > 0:
        this_sgRNA_y_start = -1*num_sgRNA_rows
        this_sgRNA_y_height = num_sgRNA_rows - 0.3

        # Normalize extend_left_non_gap to a list the length of sgRNA_intervals
        if extend_left_non_gap is None:
            extend = [0]*len(sgRNA_intervals)
        elif isinstance(extend_left_non_gap, int):
            extend = [extend_left_non_gap]*len(sgRNA_intervals)
        elif isinstance(extend_left_non_gap, dict):
            # allow mapping by index or (if available) name
            extend = [0]*len(sgRNA_intervals)
            for i in range(len(sgRNA_intervals)):
                extend[i] = extend_left_non_gap.get(
                    sgRNA_names[i] if sgRNA_names and i < len(sgRNA_names) else i,
                    extend_left_non_gap.get(i, 0)
                )
        else:
            extend = list(extend_left_non_gap)
            if len(extend) < len(sgRNA_intervals):
                extend += [0]*(len(sgRNA_intervals)-len(extend))
            else:
                extend = extend[:len(sgRNA_intervals)]

        add_sgRNA_to_ax(
            ax_hm_ref,
            sgRNA_intervals,
            sgRNA_y_start=this_sgRNA_y_start,
            sgRNA_y_height=this_sgRNA_y_height,
            amp_len=plot_nuc_len,
            font_size='small',
            clip_on=False,
            sgRNA_names=sgRNA_names,
            sgRNA_mismatches=sgRNA_mismatches,
            x_offset=0,
            label_at_zero=True,
            sgRNA_rows=sgRNA_rows,
            sgRNA_label_sides=['left','right'],
            ref_row_seq=ref_aln_seq_region,     # WT ref alignment row (with '-')
            extend_left_non_gap=extend          # apply caller-provided extension
        )
    #### End of Newer method ####

    #create boxes for ins
    for idx, lss in insertion_dict.items():
        for ls in lss:
            ax_hm.add_patch(patches.Rectangle((ls[0], N_ROWS-idx-1), ls[1]-ls[0], 1, linewidth=3, edgecolor='r', fill=False))

    #### Original method
    # #cut point vertical line
    # if plot_cut_point:
    #     if cut_point_ind is None:
    #         cut_point_ind = [plot_nuc_len / 2]
    #     # ax_hm.vlines(cut_point_ind,*ax_hm.get_ylim(),linestyles='dashed')
    #     ax_hm.vlines(180, 3, N_ROWS, linestyles='dashed')

    #### New method
    #cut point vertical line
    # if plot_cut_point:
    #     positions = cut_point_ind
    #     if positions is None:
    #         positions = [plot_nuc_len / 2]
    #     elif not isinstance(positions, (list, tuple, np.ndarray)):
    #         positions = [positions]
    #     ymin, ymax = ax_hm.get_ylim()
    #     for pos in positions:
    #         ax_hm.vlines(pos, ymin, ymax, linestyles='dashed')
    #         print(pos)

#### Newer method to draw cut point lines at correct position and for WT and TwinPE refs rows ####
    # cut point vertical line
    if plot_cut_point:
        if cut_point_ind is None:
            cut_point_ind = [plot_nuc_len / 2]

        def _is_blank_row(chars):
            return all(c == " " for c in chars)

        N_ROWS = len(annot)

        n_bottom_blank = 0
        for i in range(1, min(3, N_ROWS)):
            if _is_blank_row(annot[i]):
                n_bottom_blank += 1
            else:
                break

        # Convert base indices to boundary positions (draw line AFTER the specified base).
        # If cut_point_ind values are 0-based indices of the cut base, we add +1 so the
        # vertical line appears between base i and i+1 (user expectation).
        raw_points = cut_point_ind if isinstance(cut_point_ind, (list, tuple, np.ndarray)) else [cut_point_ind]
        xs = [cp + 1 for cp in raw_points if cp is not None]

        # On allele table (ax_hm): draw over TwinPE row
        if N_ROWS >= 1:
            ax_hm.vlines(xs, 0, 1, linestyles="dashed")

        # Skip the blank spacer rows and draw through remaining allele rows
        y_start = 1 + n_bottom_blank
        if N_ROWS > y_start:
            ax_hm.vlines(xs, y_start, N_ROWS, linestyles="dashed")

        # On WT Reference (ax_hm_ref): single row
        ax_hm_ref.vlines(xs, 0, 1, linestyles="dashed")
    #### End of New method ####
    ax_hm_ref.yaxis.tick_right()
    ax_hm_ref.xaxis.set_ticks([])
    ax_hm_ref.yaxis.set_ticklabels(['WT Reference'], rotation=True, va='center')

    gs2.update(left=0, right=1, hspace=0.05, wspace=0, top=1*(((N_ROWS)*1.13))/(N_ROWS))
    gs1.update(left=0, right=1, hspace=0.05, wspace=0,)

    sns.set_context(rc={'axes.facecolor':'white','lines.markeredgewidth': 1,'mathtext.fontset' : 'stix','text.usetex':True,'text.latex.unicode':True} )

    proxies = [matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='black',
                    mec='none', marker=r'$\mathbf{{{}}}$'.format('bold'), ms=18),
               matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='none',
                    mec='r', marker='s', ms=8, markeredgewidth=2.5),
              matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='none',
                    mec='black', marker='_', ms=2,)]
    descriptions=['Substitutions', 'Insertions', 'Deletions']

    if plot_cut_point:
        proxies.append(
              matplotlib.lines.Line2D([0], [1], linestyle='--', c='black', ms=6))
        descriptions.append('Predicted cleavage position')

    if category:
        category = category.replace("_", r"\ ")
        proxies.append(patches.Patch(color='none'))
        descriptions.append(rf"$\bf{{{category}\ Alleles}}$")

    #ax_hm_ref.legend(proxies, descriptions, numpoints=1, markerscale=2, loc='center', bbox_to_anchor=(0.5, 4),ncol=1)
    lgd = ax_hm.legend(proxies, descriptions, numpoints=1, markerscale=2, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=1, fancybox=True, shadow=False)

    if fig_filename_root is None:
        plt.show()
    else:
        fig.savefig(fig_filename_root+'.pdf', bbox_inches='tight', bbox_extra_artists=(lgd,))
        if SAVE_ALSO_PNG:
            fig.savefig(fig_filename_root+'.png', bbox_inches='tight', bbox_extra_artists=(lgd,))
    plt.close(fig)


# def plot_alleles_heatmap(
#         reference_seq,
#         twin_aln_seq,
#         X,
#         annot,
#         y_labels,
#         insertion_dict,
#         per_element_annot_kws,
#         fig_filename_root=None,
#         custom_colors=None,
#         SAVE_ALSO_PNG=False,
#         plot_cut_point=True,
#         cut_point_ind=None,
#         sgRNA_intervals=None,
#         sgRNA_names=None,
#         sgRNA_mismatches=None,
#         category=None,
#         **kwargs):
#     """
#     Render a nucleotide-annotated heatmap of allele sequences with an optional reference
#     row, optional sgRNA overlays, and optional cleavage position line.

#     Args:
#         reference_seq (str): Ungapped reference sequence shown as the top (reference) row.
#         X (list[list[int]] | np.ndarray): Matrix of numeric-encoded nucleotides per allele row.
#         annot (list[list[str]] | np.ndarray): Matrix of characters for per-cell annotation
#             (same shape/order as X).
#         y_labels (list[str]): Labels for each allele row in X (rendered along the y-axis).
#         insertion_dict (dict[int, list[tuple[int,int]]]): For each row index, spans to outline
#             as insertions (drawn as red boxes around the cells covering the span).
#         per_element_annot_kws (np.ndarray): Per-cell dicts of Text properties for annotation styling.
#         fig_filename_root (str | None): If None, display interactively; else save as <root>.pdf
#             (and optionally <root>.png when SAVE_ALSO_PNG is True).
#         custom_colors (dict | None): Optional mapping like {'A': '#hex', 'T': '#hex', 'C': '#hex',
#             'G': '#hex', 'N': '#hex'}; alpha is appended automatically (~40%).
#         SAVE_ALSO_PNG (bool): If True and fig_filename_root is set, also write PNG alongside PDF.
#         plot_cut_point (bool): If True, draw a vertical dashed line at cut_point_ind (or center).
#         cut_point_ind (int | list[int] | None): One or more x indices for predicted cleavage.
#             If None, defaults to the midpoint of the reference length.
#         sgRNA_intervals (list[tuple[int,int]] | None): Optional sgRNA spans to overlay above the
#             reference row.
#         sgRNA_names (list[str] | None): Optional per-sgRNA labels.
#         sgRNA_mismatches (list[list[int]] | None): Optional per-sgRNA mismatch positions (relative
#             to the sgRNA start) to draw in red.
#         **kwargs: Forwarded to the underlying pcolormesh via custom_heatmap (e.g., shading).

#     Returns:
#         None. Displays or saves the figure and closes it.
#     """
#     plot_nuc_len=len(reference_seq)

#     # make a color map of fixed colors
#     alpha=0.4
#     A_color=get_nuc_color('A', alpha)
#     T_color=get_nuc_color('T', alpha)
#     C_color=get_nuc_color('C', alpha)
#     G_color=get_nuc_color('G', alpha)
#     INDEL_color = get_nuc_color('N', alpha)

#     # Allow overriding base colors with hex strings; append hex alpha for ~40% transparency
#     if custom_colors is not None:
#         hex_alpha = '66'  # this is equivalent to 40% in hexadecimal
#         if 'A' in custom_colors:
#             A_color = custom_colors['A'] + hex_alpha
#         if 'T' in custom_colors:
#             T_color = custom_colors['T'] + hex_alpha
#         if 'C' in custom_colors:
#             C_color = custom_colors['C'] + hex_alpha
#         if 'G' in custom_colors:
#             G_color = custom_colors['G'] + hex_alpha
#         if 'N' in custom_colors:
#             INDEL_color = custom_colors['N'] + hex_alpha

#     dna_to_numbers={'-':0,'A':1,'T':2,'C':3,'G':4,'N':5}
#     seq_to_numbers= lambda seq: [dna_to_numbers[x] for x in seq]

#     # Build the ListedColormap using the ordered mapping; the last 'N' duplicates the first to
#     # ensure gaps/ambiguous map consistently.
#     cmap = colors_mpl.ListedColormap([INDEL_color, A_color, T_color, C_color, G_color, INDEL_color])

#     #ref_seq_around_cut=reference_seq[max(0,cut_point-plot_nuc_len/2+1):min(len(reference_seq),cut_point+plot_nuc_len/2+1)]

# #    print('per element anoot kws: ' + per_element_annot_kws)
#     if len(per_element_annot_kws) > 1:
#         # Stack per-row annotation styles and invert order to align with inverted X below
#         per_element_annot_kws=np.vstack(per_element_annot_kws[::-1])
#     else:
#         per_element_annot_kws=np.array(per_element_annot_kws)
#     ref_seq_hm=np.expand_dims(seq_to_numbers(reference_seq), 1).T
#     ref_seq_annot_hm=np.expand_dims(list(reference_seq), 1).T
#     twin_seq_hm=np.expand_dims(seq_to_numbers(twin_aln_seq), 1).T
#     twin_seq_annot_hm=np.expand_dims(list(twin_aln_seq), 1).T

#     # Reverse to plot the most frequent allele at the top of the heatmap
#     annot=annot[::-1]
#     X=X[::-1]

#     N_ROWS=len(X)
#     N_COLUMNS=plot_nuc_len

#     if N_ROWS < 1:
#         # No alleles to show; emit a placeholder figure
#         fig, ax = plt.subplots()
#         fig.text(0.5, 0.5, 'No Alleles', horizontalalignment='center', verticalalignment='center', transform = ax.transAxes)
#         ax.set_clip_on(False)

#         if fig_filename_root is None:
#             plt.show()
#         else:
#             fig.savefig(fig_filename_root+'.pdf', bbox_inches='tight')
#             if SAVE_ALSO_PNG:
#                 fig.savefig(fig_filename_root+'.png', bbox_inches='tight')
#         plt.close(fig)
#         return

# # ...existing code...
#     # Determine how much vertical space is needed to accommodate optional sgRNA rows
#     sgRNA_rows = []
#     num_sgRNA_rows = 0
#     if sgRNA_intervals and len(sgRNA_intervals) > 0:
#         sgRNA_rows = get_rows_for_sgRNA_annotation(sgRNA_intervals, plot_nuc_len)
#         num_sgRNA_rows = max(sgRNA_rows) + 1

#     # Size the figure to satisfy square cells for the allele heatmap
#     cells_per_inch = 0.30  # horizontal size per base (tweak to taste)
#     fig_w = plot_nuc_len * cells_per_inch

#     # Give the ref axis extra room for sgRNA rows
#     ref_rows   = 1 + num_sgRNA_rows
#     hm_rows    = N_ROWS
#     twin_rows  = 1
#     total_rows = ref_rows + hm_rows + twin_rows

#     # Height to keep the central heatmap square: H = W * (total_rows / N_COLUMNS)
#     fig_h = fig_w * (total_rows / float(N_COLUMNS))

#     # Single GridSpec with proportional heights
#     fig = plt.figure(figsize=(fig_w, fig_h), constrained_layout=True)
#     gs = gridspec.GridSpec(3, N_COLUMNS, height_ratios=[ref_rows, hm_rows, twin_rows], figure=fig)

#     ax_hm_ref  = fig.add_subplot(gs[0, :])
#     ax_hm      = fig.add_subplot(gs[1, :])
#     ax_hm_twin = fig.add_subplot(gs[2, :])

#     # Draw reference row (top), allele matrix (middle), twin row (bottom)
#     custom_heatmap(ref_seq_hm,  annot=ref_seq_annot_hm,  annot_kws={'size':16}, cmap=cmap, fmt='s',
#                    ax=ax_hm_ref,  vmin=0, vmax=5, square=False)
#     custom_heatmap(X,           annot=np.array(annot),   annot_kws={'size':16}, cmap=cmap, fmt='s',
#                    ax=ax_hm,     vmin=0, vmax=5, square=True, per_element_annot_kws=per_element_annot_kws)
#     custom_heatmap(twin_seq_hm, annot=twin_seq_annot_hm, annot_kws={'size':16}, cmap=cmap, fmt='s',
#                    ax=ax_hm_twin, vmin=0, vmax=5, square=False)

#     # Expand ref ylim to show sgRNA rows inside the ref axes
#     if num_sgRNA_rows > 0:
#         ax_hm_ref.set_ylim(-num_sgRNA_rows, 1)

#     # Remove previous aspect/anchor/layout hacks
#     # if hasattr(ax_hm, "set_box_aspect"): ...
#     # gs.update(...)

#     # Y on the right to keep allele labels near the matrix; hide x ticks (letters are the labels)
#     ax_hm.yaxis.tick_right()
#     ax_hm.yaxis.set_ticklabels(y_labels[::-1], rotation=True, va='center')
#     ax_hm.xaxis.set_ticks([])

#     # Draw sgRNA overlays above the ref row (now there is real space in the axes)
#     if sgRNA_intervals and len(sgRNA_intervals) > 0:
#         this_sgRNA_y_start = -1*num_sgRNA_rows
#         this_sgRNA_y_height = num_sgRNA_rows - 0.3
#         add_sgRNA_to_ax(ax_hm_ref, sgRNA_intervals,
#                         sgRNA_y_start=this_sgRNA_y_start,
#                         sgRNA_y_height=this_sgRNA_y_height,
#                         amp_len=plot_nuc_len,
#                         font_size='small',
#                         clip_on=True,  # now safe to clip because ylim was expanded
#                         sgRNA_names=sgRNA_names,
#                         sgRNA_mismatches=sgRNA_mismatches,
#                         x_offset=0,
#                         label_at_zero=True,
#                         sgRNA_rows=sgRNA_rows)

#     # Reference axis cosmetics
#     ax_hm_ref.yaxis.tick_right()
#     ax_hm_ref.xaxis.set_ticks([])
#     ax_hm_ref.yaxis.set_ticklabels(['WT Reference'], rotation=True, va='center')

#     # Twin seq axis cosmetics
#     ax_hm_twin.yaxis.tick_right()
#     ax_hm_twin.xaxis.set_ticks([])
#     ax_hm_twin.yaxis.set_ticklabels(['TwinPE Reference'], rotation=True, va='center')
# # ...existing code...

#     # Draw sgRNA overlays just above the reference row when provided
#     if sgRNA_intervals and len(sgRNA_intervals) > 0:
#         this_sgRNA_y_start = -1*num_sgRNA_rows
#         this_sgRNA_y_height = num_sgRNA_rows - 0.3
#         add_sgRNA_to_ax(ax_hm_ref, sgRNA_intervals, sgRNA_y_start=this_sgRNA_y_start, sgRNA_y_height=this_sgRNA_y_height, amp_len=plot_nuc_len, font_size='small', clip_on=False, sgRNA_names=sgRNA_names, sgRNA_mismatches=sgRNA_mismatches, x_offset=0, label_at_zero=True, sgRNA_rows=sgRNA_rows)

#     # Outline insertion spans in red for each allele row
#     for idx, lss in insertion_dict.items():
#         for ls in lss:
#             ax_hm.add_patch(patches.Rectangle((ls[0], N_ROWS-idx-1), ls[1]-ls[0], 1, linewidth=3, edgecolor='r', fill=False))

#     # Draw predicted cleavage site(s) as vertical dashed line(s)
#     if plot_cut_point:
#         if cut_point_ind is None:
#             cut_point_ind = [plot_nuc_len / 2]
#         ax_hm.vlines(cut_point_ind,*ax_hm.get_ylim(),linestyles='dashed')

#     # Reference axis cosmetics
#     ax_hm_ref.yaxis.tick_right()
#     ax_hm_ref.xaxis.set_ticks([])
#     ax_hm_ref.yaxis.set_ticklabels(['WT Reference'], rotation=True, va='center')
    
#     # Twin seq axis cosmetics
#     ax_hm_twin.yaxis.tick_right()
#     ax_hm_twin.xaxis.set_ticks([])
#     ax_hm_twin.yaxis.set_ticklabels(['TwinPE Reference'], rotation=True, va='center')

#     # Tighten spacing; keep top margin roughly constant across different N_ROWS
#     # gs2.update(left=0, right=1, hspace=0.05, wspace=0, top=1*(((N_ROWS)*1.13))/(N_ROWS))
#     # gs1.update(left=0, right=1, hspace=0.05, wspace=0,)
#     # gs3.update(left=0, right=1, hspace=0.05, wspace=0,)

#     # Styling context for consistent fonts and math text
#     sns.set_context(rc={'axes.facecolor':'white','lines.markeredgewidth': 1,'mathtext.fontset' : 'stix','text.usetex':True,'text.latex.unicode':True} )

#     # Legend proxies to communicate substitution/indel encodings
#     proxies = [matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='black',
#                     mec='none', marker=r'$\mathbf{{{}}}$'.format('bold'), ms=18),
#                matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='none',
#                     mec='r', marker='s', ms=8, markeredgewidth=2.5),
#               matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='none',
#                     mec='black', marker='_', ms=2,)]
#     descriptions=['Substitutions', 'Insertions', 'Deletions']

#     if plot_cut_point:
#         proxies.append(
#               matplotlib.lines.Line2D([0], [1], linestyle='--', c='black', ms=6))
#         descriptions.append('Predicted cleavage position')

#     if category:
#         category = category.replace("_", r"\ ")
#         proxies.append(patches.Patch(color='none'))
#         descriptions.append(rf"$\bf{{{category}\ Alleles}}$")


#     # Place legend under the twin seq heatmap
#     #ax_hm_ref.legend(proxies, descriptions, numpoints=1, markerscale=2, loc='center', bbox_to_anchor=(0.5, 4),ncol=1)
#     lgd = ax_hm_twin.legend(proxies, descriptions, numpoints=1, markerscale=2, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=1, fancybox=True, shadow=False)

#     # # Place title above the entire figure
#     # if category:
#     #     fig.suptitle(f"{category} Alleles", fontsize=24, y=0.5, x=0.25)

#     # Display or save
#     if fig_filename_root is None:
#         plt.show()
#     else:
#         fig.savefig(fig_filename_root+'.pdf', bbox_inches='tight', bbox_extra_artists=(lgd,))
#         if SAVE_ALSO_PNG:
#             fig.savefig(fig_filename_root+'.png', bbox_inches='tight', bbox_extra_artists=(lgd,))
#     plt.close(fig)

# def plot_alleles_heatmap(
#         reference_seq,
#         twin_aln_seq,
#         X,
#         annot,
#         y_labels,
#         insertion_dict,
#         per_element_annot_kws,
#         fig_filename_root=None,
#         custom_colors=None,
#         SAVE_ALSO_PNG=False,
#         plot_cut_point=True,
#         cut_point_ind=None,
#         sgRNA_intervals=None,
#         sgRNA_names=None,
#         sgRNA_mismatches=None,
#         category=None,
#         **kwargs):
#     """
#     Render a nucleotide-annotated heatmap of allele sequences with an optional reference
#     row, optional sgRNA overlays, and optional cleavage position line.

#     Args:
#         reference_seq (str): Ungapped reference sequence shown as the top (reference) row.
#         X (list[list[int]] | np.ndarray): Matrix of numeric-encoded nucleotides per allele row.
#         annot (list[list[str]] | np.ndarray): Matrix of characters for per-cell annotation
#             (same shape/order as X).
#         y_labels (list[str]): Labels for each allele row in X (rendered along the y-axis).
#         insertion_dict (dict[int, list[tuple[int,int]]]): For each row index, spans to outline
#             as insertions (drawn as red boxes around the cells covering the span).
#         per_element_annot_kws (np.ndarray): Per-cell dicts of Text properties for annotation styling.
#         fig_filename_root (str | None): If None, display interactively; else save as <root>.pdf
#             (and optionally <root>.png when SAVE_ALSO_PNG is True).
#         custom_colors (dict | None): Optional mapping like {'A': '#hex', 'T': '#hex', 'C': '#hex',
#             'G': '#hex', 'N': '#hex'}; alpha is appended automatically (~40%).
#         SAVE_ALSO_PNG (bool): If True and fig_filename_root is set, also write PNG alongside PDF.
#         plot_cut_point (bool): If True, draw a vertical dashed line at cut_point_ind (or center).
#         cut_point_ind (int | list[int] | None): One or more x indices for predicted cleavage.
#             If None, defaults to the midpoint of the reference length.
#         sgRNA_intervals (list[tuple[int,int]] | None): Optional sgRNA spans to overlay above the
#             reference row.
#         sgRNA_names (list[str] | None): Optional per-sgRNA labels.
#         sgRNA_mismatches (list[list[int]] | None): Optional per-sgRNA mismatch positions (relative
#             to the sgRNA start) to draw in red.
#         **kwargs: Forwarded to the underlying pcolormesh via custom_heatmap (e.g., shading).

#     Returns:
#         None. Displays or saves the figure and closes it.
#     """
#     plot_nuc_len = len(reference_seq)

#     # make a color map of fixed colors
#     alpha = 0.4
#     A_color = get_nuc_color('A', alpha)
#     T_color = get_nuc_color('T', alpha)
#     C_color = get_nuc_color('C', alpha)
#     G_color = get_nuc_color('G', alpha)
#     INDEL_color = get_nuc_color('N', alpha)

#     # Allow overriding base colors with hex strings; append hex alpha for ~40% transparency
#     if custom_colors is not None:
#         hex_alpha = '66'  # this is equivalent to 40% in hexadecimal
#         if 'A' in custom_colors:
#             A_color = custom_colors['A'] + hex_alpha
#         if 'T' in custom_colors:
#             T_color = custom_colors['T'] + hex_alpha
#         if 'C' in custom_colors:
#             C_color = custom_colors['C'] + hex_alpha
#         if 'G' in custom_colors:
#             G_color = custom_colors['G'] + hex_alpha
#         if 'N' in custom_colors:
#             INDEL_color = custom_colors['N'] + hex_alpha

#     dna_to_numbers = {'-':0,'A':1,'T':2,'C':3,'G':4,'N':5}
#     seq_to_numbers = lambda seq: [dna_to_numbers[x] for x in seq]

#     # Build the ListedColormap using the ordered mapping; the last 'N' duplicates the first to
#     # ensure gaps/ambiguous map consistently.
#     cmap = colors_mpl.ListedColormap([INDEL_color, A_color, T_color, C_color, G_color, INDEL_color])

#     # Prepare per-element styles
#     if len(per_element_annot_kws) > 1:
#         per_element_annot_kws = np.vstack(per_element_annot_kws[::-1])
#     else:
#         per_element_annot_kws = np.array(per_element_annot_kws)

#     # Heatmap matrices for reference/twin (1 x N) and alleles (N_ROWS x N_COLUMNS)
#     ref_seq_hm        = np.expand_dims(seq_to_numbers(reference_seq), 1).T
#     ref_seq_annot_hm  = np.expand_dims(list(reference_seq), 1).T
#     twin_seq_hm       = np.expand_dims(seq_to_numbers(twin_aln_seq), 1).T
#     twin_seq_annot_hm = np.expand_dims(list(twin_aln_seq), 1).T

#     # Reverse to plot the most frequent allele at the top of the heatmap
#     annot = annot[::-1]
#     X     = X[::-1]

#     N_ROWS    = len(X)
#     N_COLUMNS = plot_nuc_len

#     if N_ROWS < 1:
#         # No alleles to show; emit a placeholder figure
#         fig, ax = plt.subplots()
#         fig.text(0.5, 0.5, 'No Alleles', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
#         ax.set_clip_on(False)

#         if fig_filename_root is None:
#             plt.show()
#         else:
#             fig.savefig(fig_filename_root + '.pdf')
#             if SAVE_ALSO_PNG:
#                 fig.savefig(fig_filename_root + '.png')
#         plt.close(fig)
#         return

#     # Determine how much vertical space is needed to accommodate optional sgRNA rows
#     sgRNA_rows = []
#     num_sgRNA_rows = 0
#     if sgRNA_intervals and len(sgRNA_intervals) > 0:
#         sgRNA_rows = get_rows_for_sgRNA_annotation(sgRNA_intervals, plot_nuc_len)
#         num_sgRNA_rows = int(max(sgRNA_rows) + 1)

#     # Figure size and GridSpec: keep allele cells square via box aspect; keep equal widths via sharex
#     cells_per_inch = 0.30  # width per base
#     fig_w = plot_nuc_len * cells_per_inch

#     ref_rows    = 1 + num_sgRNA_rows
#     hm_rows     = N_ROWS
#     twin_rows   = 1
#     legend_rows = 0.8  # small strip for legend
#     total_rows  = ref_rows + hm_rows + twin_rows + legend_rows

#     # Height so that a unit of x (one base) equals a unit of y (one row) in ax_hm:
#     # fig_h = fig_w * (total_rows / N_COLUMNS)
#     fig_h = fig_w * (total_rows / float(N_COLUMNS))

#     # Single GridSpec; no constrained_layout (avoids "collapsed to zero" warnings)
#     fig = plt.figure(figsize=(fig_w, fig_h))
#     gs = gridspec.GridSpec(
#         4, N_COLUMNS,
#         height_ratios=[ref_rows, hm_rows, twin_rows, legend_rows],
#         figure=fig
#     )

#     # Axes with shared x to enforce identical widths and x-lims
#     ax_hm      = fig.add_subplot(gs[1, :])
#     ax_hm_ref  = fig.add_subplot(gs[0, :], sharex=ax_hm)
#     ax_hm_twin = fig.add_subplot(gs[2, :], sharex=ax_hm)
#     ax_lgd     = fig.add_subplot(gs[3, :])
#     ax_lgd.axis('off')

#     # Draw heatmaps (square=False here; we'll enforce aspect via set_box_aspect)
#     custom_heatmap(ref_seq_hm,  annot=ref_seq_annot_hm,  annot_kws={'size':16}, cmap=cmap, fmt='s',
#                    ax=ax_hm_ref,  vmin=0, vmax=5, square=False)
#     custom_heatmap(X,           annot=np.array(annot),   annot_kws={'size':16}, cmap=cmap, fmt='s',
#                    ax=ax_hm,     vmin=0, vmax=5, square=False, per_element_annot_kws=per_element_annot_kws)
#     custom_heatmap(twin_seq_hm, annot=twin_seq_annot_hm, annot_kws={'size':16}, cmap=cmap, fmt='s',
#                    ax=ax_hm_twin, vmin=0, vmax=5, square=False)

#     # Make axes boxes respect data aspect (square cells) without shrinking width
#     if hasattr(ax_hm, "set_box_aspect"):
#         ax_hm.set_box_aspect(hm_rows / float(N_COLUMNS))
#         ax_hm_ref.set_box_aspect(ref_rows / float(N_COLUMNS))
#         ax_hm_twin.set_box_aspect(twin_rows / float(N_COLUMNS))
#     else:
#         # Fallback for older Matplotlib: only set equal on the main heatmap
#         ax_hm.set_aspect("equal", adjustable="box")

#     # Expand ref ylim to show sgRNA rows inside the ref axes
#     if num_sgRNA_rows > 0:
#         ax_hm_ref.set_ylim(-num_sgRNA_rows, 1)

#     # Axes cosmetics
#     ax_hm.yaxis.tick_right()
#     ax_hm.yaxis.set_ticklabels(y_labels[::-1], rotation=True, va='center')
#     ax_hm.xaxis.set_ticks([])

#     ax_hm_ref.yaxis.tick_right()
#     ax_hm_ref.xaxis.set_ticks([])
#     ax_hm_ref.yaxis.set_ticklabels(['WT Reference'], rotation=True, va='center')

#     ax_hm_twin.yaxis.tick_right()
#     ax_hm_twin.xaxis.set_ticks([])
#     ax_hm_twin.yaxis.set_ticklabels(['TwinPE Reference'], rotation=True, va='center')

#     # Draw sgRNA overlays above the reference row
#     if sgRNA_intervals and len(sgRNA_intervals) > 0:
#         this_sgRNA_y_start  = -1 * num_sgRNA_rows
#         this_sgRNA_y_height = num_sgRNA_rows - 0.3
#         add_sgRNA_to_ax(ax_hm_ref, sgRNA_intervals,
#                         sgRNA_y_start=this_sgRNA_y_start,
#                         sgRNA_y_height=this_sgRNA_y_height,
#                         amp_len=plot_nuc_len,
#                         font_size='small',
#                         clip_on=True,
#                         sgRNA_names=sgRNA_names,
#                         sgRNA_mismatches=sgRNA_mismatches,
#                         x_offset=0,
#                         label_at_zero=True,
#                         sgRNA_rows=sgRNA_rows)

#     # Outline insertion spans in red for each allele row
#     for idx_row, lss in insertion_dict.items():
#         for ls in lss:
#             ax_hm.add_patch(
#                 patches.Rectangle(
#                     (ls[0], N_ROWS - idx_row - 1),
#                     ls[1] - ls[0],
#                     1,
#                     linewidth=3,
#                     edgecolor='r',
#                     fill=False
#                 )
#             )

#     # Draw predicted cleavage site(s) as vertical dashed line(s)
#     if plot_cut_point:
#         if cut_point_ind is None:
#             cut_point_ind = [plot_nuc_len / 2]
#         ax_hm.vlines(cut_point_ind, *ax_hm.get_ylim(), linestyles='dashed')

#     # Styling context (unchanged)
#     sns.set_context(rc={'axes.facecolor':'white','lines.markeredgewidth': 1,'mathtext.fontset' : 'stix','text.usetex':True,'text.latex.unicode':True} )

#     # Legend proxies to communicate substitution/indel encodings
#     proxies = [matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='black',
#                     mec='none', marker=r'$\mathbf{{{}}}$'.format('bold'), ms=18),
#                matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='none',
#                     mec='r', marker='s', ms=8, markeredgewidth=2.5),
#                matplotlib.lines.Line2D([0], [0], linestyle='none', mfc='none',
#                     mec='black', marker='_', ms=2,)]
#     descriptions = ['Substitutions', 'Insertions', 'Deletions']

#     if plot_cut_point:
#         proxies.append(matplotlib.lines.Line2D([0], [1], linestyle='--', c='black', ms=6))
#         descriptions.append('Predicted cleavage position')

#     if category:
#         category_disp = category.replace("_", r"\ ")
#         proxies.append(patches.Patch(color='none'))
#         descriptions.append(rf"$\bf{{{category_disp}\ Alleles}}$")

#     # Legend in its own row to avoid bbox_inches='tight' layout side-effects
#     ax_lgd.legend(proxies, descriptions, numpoints=1, markerscale=2,
#                   loc='center', ncol=1, fancybox=True, shadow=False)

#     # Save/show without constrained/tight layout to keep axes widths identical
#     if fig_filename_root is None:
#         plt.show()
#     else:
#         fig.savefig(fig_filename_root + '.pdf')
#         if SAVE_ALSO_PNG:
#             fig.savefig(fig_filename_root + '.png')
#     plt.close(fig)

In [23]:
# def get_dataframe_allele_region(df_alleles, pegRNA_intervals, ref_seq, ref_aln_seq, twin_aln_seq, cut_points=None):
#     """
#     Extract region of interest for plotting allele tables based on pegRNA positions on reference sequence.
#     Returns modified reference sequence and cut points.
#     """
#     if df_alleles.shape[0] == 0:
#         return df_alleles
#     if len(pegRNA_intervals) == 2:
#         left_end = max(0, pegRNA_intervals[0][0] - 20)
#         right_end = min(len(ref_seq), pegRNA_intervals[1][1] + 20)
#         cut_points[0] = cut_points[0] - left_end
#         cut_points[1] = cut_points[1] - left_end
#     elif len(pegRNA_intervals) == 1:
#         left_end = max(0, pegRNA_intervals[0][0] - 120)
#         right_end = min(len(ref_seq), pegRNA_intervals[0][1] + 120)
#         # cut_points = pegRNA_intervals[0][1] - 3
 
    
#     # Don’t overwrite the original
#     df = df_alleles.copy()

#     # Slice the two sequence columns only
#     df['Aligned_Sequence']   = df['Aligned_Sequence'].str.slice(left_end, right_end)
#     df['Reference_Sequence'] = df['Reference_Sequence'].str.slice(left_end, right_end)

#     # Aggregate counts by sequence + metadata
#     df = (
#         df.groupby(['Aligned_Sequence', 'Reference_Sequence', 'Read_Status',
#                     'n_deleted', 'n_inserted', 'n_mutated'], dropna=False)
#           .sum()
#           .reset_index()
#           .set_index('Aligned_Sequence')
#     )

#     df.sort_values(by=['#Reads', 'Aligned_Sequence', 'Reference_Sequence'],
#                    ascending=[False, True, True],
#                    inplace=True)

#     df['Unedited'] = df['Read_Status'].eq('UNMODIFIED')

#     return df, ref_seq[left_end:right_end], ref_aln_seq[left_end:right_end], twin_aln_seq[left_end:right_end], cut_points

# ...existing code...
def get_dataframe_allele_region(df_alleles, pegRNA_intervals, ref_seq, ref_aln_seq, twin_aln_seq, cut_points=None):
    """
    Extract region of interest for plotting allele tables based on pegRNA positions on reference sequence.
    Returns: (df_window, ref_seq_window, ref_aln_seq_window, twin_aln_seq_window, cut_points_window, pegRNA_intervals_window)
    All returned coordinates (cut points, intervals) are shifted to the window (0-based) coordinates.
    """
    if df_alleles.shape[0] == 0:
        # Return empty frame and pass-through sequences, with None for coords
        return df_alleles, ref_seq, ref_aln_seq, twin_aln_seq, cut_points, pegRNA_intervals

    # Default padding
    if len(pegRNA_intervals) == 2:
        # left_end  = max(0, pegRNA_intervals[0][0] - 20)
        left_end  = max(0, pegRNA_intervals[0][0] - 20)
        right_end = min(len(ref_seq), pegRNA_intervals[1][1] + 20)
    elif len(pegRNA_intervals) == 1:
        # left_end  = max(0, pegRNA_intervals[0][0] - 120)
        left_end  = 0
        # right_end = min(len(ref_seq), pegRNA_intervals[0][1] + 120)
        right_end = len(ref_seq)-1
    else:
        # Fallback: show full sequence
        left_end, right_end = 0, len(ref_seq)

    # Shift cut points into window coords (if provided)
    cut_points_window = None
    if cut_points is not None:
        try:
            cut_points_window = [cp - left_end for cp in cut_points]
        except TypeError:
            # Single integer
            cut_points_window = [cut_points - left_end]

    # Shift sgRNA intervals into window coords
    pegRNA_intervals_window = []
    if pegRNA_intervals is not None:
        for (s, e) in pegRNA_intervals:
            pegRNA_intervals_window.append((s - left_end, e - left_end))

    # Don’t overwrite the original
    df = df_alleles.copy()

    # Slice the two sequence columns only
    df['Aligned_Sequence']   = df['Aligned_Sequence'].str.slice(left_end, right_end)
    df['Reference_Sequence'] = df['Reference_Sequence'].str.slice(left_end, right_end)

    # Aggregate counts by sequence + metadata
    df = (
        df.groupby(['Aligned_Sequence', 'Reference_Sequence', 'Read_Status',
                    'n_deleted', 'n_inserted', 'n_mutated'], dropna=False)
          .sum()
          .reset_index()
          .set_index('Aligned_Sequence')
    )

    df.sort_values(by=['#Reads', 'Aligned_Sequence', 'Reference_Sequence'],
                   ascending=[False, True, True],
                   inplace=True)

    df['Unedited'] = df['Read_Status'].eq('UNMODIFIED')

    return (
        df,
        ref_seq[left_end:right_end],
        ref_aln_seq[left_end:right_end],
        twin_aln_seq[left_end:right_end],
        cut_points_window,
        pegRNA_intervals_window
    )
# ...existing code...

In [24]:
def setAlleleMatplotlibDefaults():
    font = {'size': 22}
    matplotlib.rc('font', **font)
    matplotlib.use('AGG')
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
    matplotlib.rcParams["font.sans-serif"] = ["Arial", "Liberation Sans", "Bitstream Vera Sans"]
    matplotlib.rcParams["font.family"] = "sans-serif"
    matplotlib.rcParams['axes.facecolor'] = 'white'
    sns.set(style='white', font_scale=2.2)
    plt.ioff()

setAlleleMatplotlibDefaults()

In [25]:
# Some alleles are longer or shorter than the reference sequence
# These Allele tables plot smaller between full reference sequence and pegRNA +20bp padding reference
# Can't extend window 20 bases left of first sgRNA because bc its beyond edge of reference sequence

MIN_FREQUENCY = crispresso2_info['running_info']['args'].min_frequency_alleles_around_cut_to_plot
MAX_N_ROWS = crispresso2_info['running_info']['args'].max_rows_alleles_around_cut_to_plot

ref_name = crispresso2_info['results']['ref_names'][0]
ref = crispresso2_info['results']['refs'][ref_name]
# ref_seq = ref['sequence']

b_extra = 3 # set based on number of bases cleaved in order to find spacerB in compound ref

pegRNA_cut_points = ref['sgRNA_cut_points'].copy()
# adjust spacerB cut point
pegRNA_cut_points[1] = pegRNA_cut_points[1] - b_extra

pegRNA_plot_cut_points = ref['sgRNA_plot_cut_points']
pegRNA_intervals = ref['sgRNA_intervals']
pegRNA_mismatches = ref['sgRNA_mismatches']
pegRNA_names = ref['sgRNA_names']
if len(ref['sgRNA_sequences']) == 2 and pegRNA_names == ['', '']:
    pegRNA_names = ['pegRNA', 'pegRNA']

plot_count = 0
for cat in df_alleles['Category'].unique():
    df_alleles_cat = df_alleles[df_alleles['Category'] == cat]

    df_alleles_around_region, ref_seq_region, ref_aln_seq_region, twin_aln_seq_region, pegRNA_cut_points_region, pegRNA_intervals_region = get_dataframe_allele_region(
        df_alleles_cat, 
        pegRNA_intervals, 
        wt_seq, 
        wt_aln_seq,
        twin_aln_seq,
        pegRNA_cut_points
    )

    X, annot, y_labels, insertion_dict, per_element_annot_kws, is_reference = prep_alleles_table(
        df_alleles_around_region, 
        ref_seq_region, 
        ref_aln_seq_region, 
        twin_aln_seq_region,
        MAX_N_ROWS, 
        MIN_FREQUENCY
    )
    
    plot_alleles_heatmap(
        reference_seq=ref_aln_seq_region,
        X=X,
        annot=annot,
        y_labels=y_labels,
        insertion_dict=insertion_dict,
        per_element_annot_kws=per_element_annot_kws,
        fig_filename_root=f"figures/02_final_classify_reads_7cat/02_cat_{cat}",
        custom_colors=None,
        SAVE_ALSO_PNG=False,
        plot_cut_point=pegRNA_plot_cut_points,
        cut_point_ind=pegRNA_cut_points_region,
        sgRNA_intervals=pegRNA_intervals_region,
        sgRNA_names=pegRNA_names,
        sgRNA_mismatches=pegRNA_mismatches,
        category=cat,
        extend_left_non_gap=[0, b_extra]   # per-sgRNA non-gap extension

    )

# for cat in df_alleles['Category'].unique():
#     df_alleles_cat = df_alleles[df_alleles['Category'] == cat]

#     df_alleles_around_region, ref_seq_region, ref_aln_seq_region, twin_aln_seq_region, pegRNA_cut_points = get_dataframe_allele_region(
#         df_alleles_cat, 
#         pegRNA_intervals, 
#         ref_seq, 
#         ref_aln_seq,
#         twin_aln_seq,
#         pegRNA_cut_points
#     )

#     X, annot, y_labels, insertion_dict, per_element_annot_kws, is_reference = prep_alleles_table(
#         df_alleles_around_region, 
#         ref_seq_region, 
#         ref_aln_seq_region, 
#         twin_aln_seq_region,
#         MAX_N_ROWS, 
#         MIN_FREQUENCY
#     )

#     plot_alleles_heatmap(
#         reference_seq=ref_seq_region,
#         X=X,
#         annot=annot,
#         y_labels=y_labels,
#         insertion_dict=insertion_dict,
#         per_element_annot_kws=per_element_annot_kws,
#         fig_filename_root=f"figures/anzalone_pe3_12bp_ins/01_cat_{cat}",
#         custom_colors=None,
#         SAVE_ALSO_PNG=False,
#         plot_cut_point=pegRNA_plot_cut_points,
#         cut_point_ind=pegRNA_cut_points,
#         sgRNA_intervals=pegRNA_intervals,
#         sgRNA_names=pegRNA_names,
#         sgRNA_mismatches=pegRNA_mismatches,
#         category=cat
#     )
    plot_count += 1
    print('Plotted category: ' + str(cat))

print('Plotted ' + str(plot_count) + ' plots')

Plotted category: WT
Plotted category: Perfect_PE
Plotted category: Right_Flap
Plotted category: Imperfect_PE
Plotted category: PE_Indel
Plotted category: WT_Indel
Plotted category: Left_Flap
Plotted 7 plots
